# Computação Gráfica e Realidade Aumentada 
## &copy; 2025
<author>Authors: Guilherme Cabaço e Henrique Gomes</author>
<h1>Assignment 2</h1>

In [ ]:
%%html               
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecshader.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecapp.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgraobject.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgratexture.js"></script>
<script src='https://git.io/glm-js.min.js'></script>

### Vertex e Fragment Shader

In [ ]:
%%html
<script id="my-vertex-shader" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
uniform mat4 MVP;

varying  vec3 ex_Color;

void main(void) {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader" type="x-shader/x-fragment">
precision mediump float;

varying  vec3 ex_Color;

void main(void) {
  
    gl_FragColor = vec4(ex_Color,1.0);
}
</script>

In [ ]:
%%html
<script id="my-vertex-shader-texture" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
attribute  vec2 in_Tex_Coord;
uniform mat4 MVP;

varying  vec3 ex_Color;
varying  vec2 ex_Tex_Coord;

void main(void) {
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
    ex_Color = in_Color;
    ex_Tex_Coord = in_Tex_Coord;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader-texture" type="x-shader/x-fragment">
precision mediump float;

varying  vec3 ex_Color;
varying  vec2 ex_Tex_Coord;

uniform  sampler2D myTexture;

void main(void) {
    gl_FragColor = texture2D(myTexture, ex_Tex_Coord);
}
</script>

### A.1

Devido ao index buffer, foi criada uma função draw semelhante à drawit que permite a utilização do gl.drawElements

In [ ]:
%%html

<script id="cube">
class cube extends CGRAobject{
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);
        
        var vertices = [
          -0.5,-0.5,-0.5,  //0
          -0.5,-0.5, 0.5,  //1
          -0.5, 0.5,-0.5,  //2
          -0.5, 0.5, 0.5,  //3
           0.5,-0.5,-0.5,  //4
           0.5,-0.5, 0.5,  //5
           0.5, 0.5,-0.5,  //6
           0.5, 0.5, 0.5   //7
        ];

        var indices = [
          1,5,7, 1,7,3,  // front
          0,2,6, 0,6,4,  // back
          0,1,3, 0,3,2,  // left
          4,6,7, 4,7,5,  // right
          2,3,7, 2,7,6,  // top
          0,4,5, 0,5,1   // bottom
        ];

        var colors = [];
        for (let i = 0; i < vertices.length; i += 3) {//check x of every vertex
            
          //If x==-0.5 assign color1 otherwise its the other side and assign color2  
          const t = (vertices[i] + 0.5);
            
          colors.push(
            color1[0]*(1-t)+color2[0]*t,
            color1[1]*(1-t)+color2[1]*t,
            color1[2]*(1-t)+color2[2]*t
          );
        }
        
        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }
    
    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>

In [ ]:
%%html

<script id="cubeT">
class cubeT extends CGRAobject{
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);
        
        var vertices = [
          -0.5,-0.5,-0.5,  //0
          -0.5,-0.5, 0.5,  //1
          -0.5, 0.5,-0.5,  //2
          -0.5, 0.5, 0.5,  //3
           0.5,-0.5,-0.5,  //4
           0.5,-0.5, 0.5,  //5
           0.5, 0.5,-0.5,  //6
           0.5, 0.5, 0.5   //7
        ];

        var indices = [
          1,5,7, 1,7,3,  // front
          0,2,6, 0,6,4,  // back
          0,1,3, 0,3,2,  // left
          4,6,7, 4,7,5,  // right
          2,3,7, 2,7,6,  // top
          0,4,5, 0,5,1   // bottom
        ];

        var colors = [];
        for (let i = 0; i < vertices.length; i += 3) {//check x of every vertex
            
          //If x==-0.5 assign color1 otherwise its the other side and assign color2  
          const t = (vertices[i] + 0.5);
            
          colors.push(
            color1[0]*(1-t)+color2[0]*t,
            color1[1]*(1-t)+color2[1]*t,
            color1[2]*(1-t)+color2[2]*t
          );
        }
        
        const texcoords = [];
        for (let i = 0; i < vertices.length; i += 3) {
          const x = vertices[i];     // -0.5 .. 0.5
          const y = vertices[i + 1]; // -0.5 .. 0.5
          const u = x + 0.5;         // 0 ou 1
          const v = y + 0.5;         // 0 ou 1
          texcoords.push(u, v);
        }
        
        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.texcoordbuffer = this.gl.createBuffer();                           
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texcoords), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }
    
    setTexture(tex){
        this.texture = tex;      
    }
    
    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        
        const texLoc = gl.getAttribLocation(shader.shaderProgram, "in_Tex_Coord");
        if (texLoc !== -1) {
            gl.bindBuffer(gl.ARRAY_BUFFER, this.texcoordbuffer);
            gl.vertexAttribPointer(texLoc, 2, gl.FLOAT, false, 0, 0);
            gl.enableVertexAttribArray(texLoc);
        }

        if (this.texture) {                                            
            gl.activeTexture(gl.TEXTURE0);
            gl.bindTexture(gl.TEXTURE_2D, this.texture.textureid);
            const samplerLoc = gl.getUniformLocation(shader.shaderProgram, "myTexture");
            if (samplerLoc !== -1) {
                gl.uniform1i(samplerLoc, 0); // usar TEXTURE0
            }
        }
        
        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>

In [ ]:
%%html

<script id="sphere">

class sphere extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);
        this.gl = glcontext;

        var nCircles = 12;  // number of circles
        var nVertices = 12;  // number of vertices per circle
        var radius = 0.5;

        var vertices = [];
        var indices = [];
        var colors = [];

        // Top vertice
        vertices.push(0.0, radius, 0.0);
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        const topIndex = 0;

        // Generate middle circles
        for (let i = 1; i < nCircles; i++) {
            
            var r = radius * Math.sin(i / nCircles * Math.PI);
            var y = radius * Math.cos(i / nCircles * Math.PI);
            
            for (let j = 0; j < nVertices; j++) {
                
                var x = r * Math.cos(j / nVertices * 2 * Math.PI);
                var z = r * Math.sin(j / nVertices * 2 * Math.PI);
                
                vertices.push(x, y, z);
                
                var t = j/nVertices; // based on x coordinate
                colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
                );
            }
        }

        // Bottom vertice
        vertices.push(0.0, -radius, 0.0);
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        const bottomIndex = vertices.length / 3 - 1;

        // Indices
        // Top 
        for (let j = 1; j < nVertices; j++) {
            const next = (j % nVertices) +1;
            indices.push(topIndex, j, next);
        }

        // Middle stacks
        for (let i = 0; i < nCircles - 2; i++) {
            var start = 1 + i * nVertices;
            var next = start + nVertices;
            
            for (let j = 0; j < nVertices; j++) {
                const k = (j % nVertices) + 1;
            
                indices.push(start + j, next + j, next + k);
                indices.push(start + j, next + k, start + k);
            }
        }

        // Bottom 
        var lastRingStart = 1 + (nCircles - 2) * nVertices;
        for (let j = 0; j < nVertices; j++) {
            var next = (j % nVertices) + 1;
            indices.push(lastRingStart + j, bottomIndex, lastRingStart + next);
        }
        

        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }

    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}


</script>

In [ ]:
%%html

<script id="sphereT">

class sphereT extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);
        this.gl = glcontext;

        var nCircles = 12;  // number of circles
        var nVertices = 12;  // number of vertices per circle
        var radius = 0.5;

        var vertices = [];
        var indices = [];
        var colors = [];
        var texcoords = [];

        // Top vertice
        vertices.push(0.0, radius, 0.0);
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        texcoords.push(0.5, 0.0);
        const topIndex = 0;

        // Generate middle circles
        for (let i = 1; i < nCircles; i++) {
            
            var r = radius * Math.sin(i / nCircles * Math.PI);
            var y = radius * Math.cos(i / nCircles * Math.PI);
            var v = i / nCircles;
            
            for (let j = 0; j < nVertices; j++) {
                
                var x = r * Math.cos(j / nVertices * 2 * Math.PI);
                var z = r * Math.sin(j / nVertices * 2 * Math.PI);
                
                vertices.push(x, y, z);
                
                var t = j/nVertices; // based on x coordinate
                colors.push(
                    color1[0] * (1 - t) + color2[0] * t,
                    color1[1] * (1 - t) + color2[1] * t,
                    color1[2] * (1 - t) + color2[2] * t
                );
                
                var u = j / nVertices;   
                texcoords.push(u, v);
            }
        }

        // Bottom vertice
        vertices.push(0.0, -radius, 0.0);
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        texcoords.push(0.5, 1.0);
        const bottomIndex = vertices.length / 3 - 1;

        // Indices
        // Top 
        for (let j = 1; j < nVertices; j++) {
            const next = (j % nVertices) +1;
            indices.push(topIndex, j, next);
        }

        // Middle stacks
        for (let i = 0; i < nCircles - 2; i++) {
            var start = 1 + i * nVertices;
            var next = start + nVertices;
            
            for (let j = 0; j < nVertices; j++) {
                const k = (j % nVertices) + 1;
            
                indices.push(start + j, next + j, next + k);
                indices.push(start + j, next + k, start + k);
            }
        }

        // Bottom 
        var lastRingStart = 1 + (nCircles - 2) * nVertices;
        for (let j = 0; j < nVertices; j++) {
            var next = (j % nVertices) + 1;
            indices.push(lastRingStart + j, bottomIndex, lastRingStart + next);
        }
        

        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        // texcoords
        this.texcoordbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texcoords), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }
    
    setTexture(tex) {
        this.texture = tex;
    }

    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        // texcoords
        const texLoc = gl.getAttribLocation(shader.shaderProgram, "in_Tex_Coord");
        if (texLoc !== -1) {
            gl.bindBuffer(gl.ARRAY_BUFFER, this.texcoordbuffer);
            gl.vertexAttribPointer(texLoc, 2, gl.FLOAT, false, 0, 0);
            gl.enableVertexAttribArray(texLoc);
        }

        // ligar textura ao sampler myTexture
        if (this.texture) {
            gl.activeTexture(gl.TEXTURE0);
            gl.bindTexture(gl.TEXTURE_2D, this.texture.textureid);
            const samplerLoc = gl.getUniformLocation(shader.shaderProgram, "myTexture");
            if (samplerLoc !== -1) {
                gl.uniform1i(samplerLoc, 0);
            }
        }
        
        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}


</script>

In [ ]:
%%html

<script id="cylinder">
class cylinder extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);

        const n = 12;
        const radius = 0.5;
        const height = 1.0;

        const vertices = [];
        const colors = [];
        const indices = [];

        //Top and base circle vertices

        for (let i = 0; i < n; i++) {

            var x = radius * Math.cos((i / n) * 2 * Math.PI);
            var z = radius * Math.sin((i / n) * 2 * Math.PI);
            vertices.push(x, -height / 2, z);

            var t = i/n; // Gradient along the vertices
            colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
            );
        }

        for (let i = 0; i < n; i++) {

            var x = radius * Math.cos((i / n) * 2 * Math.PI);
            var z = radius * Math.sin((i / n) * 2 * Math.PI);
            vertices.push(x, height / 2, z);

            var t = i/n; // Gradient along the vertices
            colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
            );
        }

        // Top and base center vertices
        var baseIndex = vertices.length / 3; //Divide by 3 because there are 3 vertices for each index
        vertices.push(0, -height / 2, 0);
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );

        var topIndex = vertices.length / 3; //Divide by 3 because there are 3 vertices for each index
        vertices.push(0, height / 2, 0);
         colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );


        //Side indices
        for (let i = 0; i < n; i++) {
            var next = (i + 1) % n; //The result is always the next number after i except in the last where we want 0 again
            var bottom1 = i;
            var bottom2 = next;
            var top1 = i + n;
            var top2 = next + n;

            // First triangle
            indices.push(bottom1, top1, bottom2);
            // Second triangle
            indices.push(bottom2, top1, top2);
        }

        // Top indices
        for (let i = 0; i < n; i++) {
            var next = (i + 1) % n; //The result is always the next number after i except in the last where we want 0 again
            indices.push(topIndex, i+n, next+n);
        }
        
        // Base indices
        for (let i = 0; i < n; i++) {
            var next = (i + 1) % n; //The result is always the next number after i except in the last where we want 0 again
            indices.push(baseIndex, next, i);
        }


        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }

    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


In [ ]:
%%html

<script id="cylinderT">
class cylinderT extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);

        const n = 12;
        const radius = 0.5;
        const height = 1.0;

        const vertices = [];
        const colors = [];
        const indices = [];
        const texcoords = [];
        
        //Top and base circle vertices

        for (let i = 0; i < n; i++) {

            var x = radius * Math.cos((i / n) * 2 * Math.PI);
            var z = radius * Math.sin((i / n) * 2 * Math.PI);
            vertices.push(x, -height / 2, z);

            var t = i/n; // Gradient along the vertices
            colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
            );
            
            const u = i / n;
            const v = 0.0;
            texcoords.push(u, v);
        }

        for (let i = 0; i < n; i++) {

            var x = radius * Math.cos((i / n) * 2 * Math.PI);
            var z = radius * Math.sin((i / n) * 2 * Math.PI);
            vertices.push(x, height / 2, z);

            var t = i/n; // Gradient along the vertices
            colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
            );
            
            const u = i / n;
            const v = 1.0;
            texcoords.push(u, v);
        }

        // Top and base center vertices
        var baseIndex = vertices.length / 3; //Divide by 3 because there are 3 vertices for each index
        vertices.push(0, -height / 2, 0);
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        texcoords.push(0.5, 0.0);

        var topIndex = vertices.length / 3; //Divide by 3 because there are 3 vertices for each index
        vertices.push(0, height / 2, 0);
         colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        texcoords.push(0.5, 1.0);


        //Side indices
        for (let i = 0; i < n; i++) {
            var next = (i + 1) % n; //The result is always the next number after i except in the last where we want 0 again
            var bottom1 = i;
            var bottom2 = next;
            var top1 = i + n;
            var top2 = next + n;

            // First triangle
            indices.push(bottom1, top1, bottom2);
            // Second triangle
            indices.push(bottom2, top1, top2);
        }

        // Top indices
        for (let i = 0; i < n; i++) {
            var next = (i + 1) % n; //The result is always the next number after i except in the last where we want 0 again
            indices.push(topIndex, i+n, next+n);
        }
        
        // Base indices
        for (let i = 0; i < n; i++) {
            var next = (i + 1) % n; //The result is always the next number after i except in the last where we want 0 again
            indices.push(baseIndex, next, i);
        }


        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        // texcoords
        this.texcoordbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texcoords), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }
    
    setTexture(tex) {
        this.texture = tex;
    }

    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        // texcoords
        const texLoc = gl.getAttribLocation(shader.shaderProgram, "in_Tex_Coord");
        if (texLoc !== -1) {
            gl.bindBuffer(gl.ARRAY_BUFFER, this.texcoordbuffer);
            gl.vertexAttribPointer(texLoc, 2, gl.FLOAT, false, 0, 0);
            gl.enableVertexAttribArray(texLoc);
        }

        // texture
        if (this.texture) {
            gl.activeTexture(gl.TEXTURE0);
            gl.bindTexture(gl.TEXTURE_2D, this.texture.textureid);
            const samplerLoc = gl.getUniformLocation(shader.shaderProgram, "myTexture");
            if (samplerLoc !== -1) {
                gl.uniform1i(samplerLoc, 0);
            }
        }
        
        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


In [ ]:
%%html

<script id="cone">

class cone extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);
        this.gl = glcontext;
        
        var n = 12;
        var radius = 0.5;
        var height = 1.0;
        var vertices = [];
        var indices = [];
        var colors = [];
        
        // top center vertice
        vertices.push(0.0, height/2.0, 0.0);
        
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        
        // base vertices
        for (let i = 0; i < n; i++) {
            var x = radius * Math.cos((i / n) * 2 * Math.PI);
            var y = -height/2;
            var z = radius * Math.sin((i / n) * 2 * Math.PI);
            vertices.push(x, y, z);
            
          var t = i/n; //Gradient along the vertices
            colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
            );
        }
        
        // base center vertice
        vertices.push(0.0, -height/2.0, 0.0);
        
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        
        var topIndex = 0;
        var baseIndex = n + 1;
        
        // Indices
        for (let i = 1; i <= n; i++) {
            var next = (i % n) + 1; //The result is always the next number after i except in the last where we want 0 again
            indices.push(topIndex, i, next);
        }
        
        for (let i = 1; i <= n; i++) {
            var next = (i % n) + 1; //The result is always the next number after i except in the last where we want 0 again
            indices.push(baseIndex, next, i);
        }

        
        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }
    
    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }

}

</script>

In [ ]:
%%html

<script id="coneT">

class coneT extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);
        this.gl = glcontext;
        
        var n = 12;
        var radius = 0.5;
        var height = 1.0;
        var vertices = [];
        var indices = [];
        var colors = [];
        var texcoords  = [];
        
        // top center vertice
        vertices.push(0.0, height/2.0, 0.0);
        
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        texcoords.push(0.5, 1.0);
        
        // base vertices
        for (let i = 0; i < n; i++) {
            var x = radius * Math.cos((i / n) * 2 * Math.PI);
            var y = -height/2;
            var z = radius * Math.sin((i / n) * 2 * Math.PI);
            vertices.push(x, y, z);
            
            var t = i/n; //Gradient along the vertices
            colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
            );

            // mapeamento de textura para a lateral
            var u = i / n;
            var v = 0.0;
            texcoords.push(u, v);
        }
        
        // base center vertice
        vertices.push(0.0, -height/2.0, 0.0);
        
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        texcoords.push(0.5, 0.0);
        
        var topIndex = 0;
        var baseIndex = n + 1;
        
        // Indices
        for (let i = 1; i <= n; i++) {
            var next = (i % n) + 1; //The result is always the next number after i except in the last where we want 0 again
            indices.push(topIndex, i, next);
        }
        
        for (let i = 1; i <= n; i++) {
            var next = (i % n) + 1; //The result is always the next number after i except in the last where we want 0 again
            indices.push(baseIndex, next, i);
        }

        
        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        // texcoords
        this.texcoordbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texcoords), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }
    
    setTexture(tex) {
        this.texture = tex;
    }
    
    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        // texcoords
        const texLoc = gl.getAttribLocation(shader.shaderProgram, "in_Tex_Coord");
        if (texLoc !== -1) {
            gl.bindBuffer(gl.ARRAY_BUFFER, this.texcoordbuffer);
            gl.vertexAttribPointer(texLoc, 2, gl.FLOAT, false, 0, 0);
            gl.enableVertexAttribArray(texLoc);
        }

        // texture
        if (this.texture) {
            gl.activeTexture(gl.TEXTURE0);
            gl.bindTexture(gl.TEXTURE_2D, this.texture.textureid);
            const samplerLoc = gl.getUniformLocation(shader.shaderProgram, "myTexture");
            if (samplerLoc !== -1) {
                gl.uniform1i(samplerLoc, 0);
            }
        }
        
        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }

}

</script>

In [ ]:
%%html

<script id="disk">
class disk extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1], n = 12) {
        super(glcontext);

        var radius = 0.5;

        var vertices = [0.0, 0.0, 0.0]; //Center vertex
        
        var colors = [];
        // Center color 
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );

        for (var i = 0; i < n; i++) {
            
            var x = radius * Math.cos((i / n) * 2 * Math.PI);
            var y = 0.0;
            var z = radius * Math.sin((i / n) * 2 * Math.PI);
            vertices.push(x, y, z);

            var t = i/n; // Gradient along the vertices
            colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
            );
        }

        var indices = [];
        for (let i = 1; i <= n; i++) {
            var next = (i % n) + 1; //The result is always the next number after i except in the last where we want 1 again
            indices.push(0, i, next);
        }

        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }

    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


In [ ]:
%%html

<script id="diskT">
class diskT extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1], n = 12) {
        super(glcontext);

        var radius = 0.5;

        var vertices = [0.0, 0.0, 0.0]; //Center vertex
        var texcoords = [];
        
        var colors = [];
        // Center color 
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        texcoords.push(0.5, 0.5);

        for (var i = 0; i < n; i++) {
            
            var x = radius * Math.cos((i / n) * 2 * Math.PI);
            var y = 0.0;
            var z = radius * Math.sin((i / n) * 2 * Math.PI);
            vertices.push(x, y, z);

            var t = i/n; // Gradient along the vertices
            colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
            );
            
            var u = x + 0.5;
            var v = z + 0.5;
            texcoords.push(u, v);
        }

        var indices = [];
        for (let i = 1; i <= n; i++) {
            var next = (i % n) + 1; //The result is always the next number after i except in the last where we want 1 again
            indices.push(0, i, next);
        }

        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        // texcoords
        this.texcoordbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texcoords), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }

    setTexture(tex) {
        this.texture = tex;
    }
    
    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        // texcoords
        const texLoc = gl.getAttribLocation(shader.shaderProgram, "in_Tex_Coord");
        if (texLoc !== -1) {
            gl.bindBuffer(gl.ARRAY_BUFFER, this.texcoordbuffer);
            gl.vertexAttribPointer(texLoc, 2, gl.FLOAT, false, 0, 0);
            gl.enableVertexAttribArray(texLoc);
        }

        // texture
        if (this.texture) {
            gl.activeTexture(gl.TEXTURE0);
            gl.bindTexture(gl.TEXTURE_2D, this.texture.textureid);
            const samplerLoc = gl.getUniformLocation(shader.shaderProgram, "myTexture");
            if (samplerLoc !== -1) {
                gl.uniform1i(samplerLoc, 0);
            }
        }
        
        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


In [ ]:
%%html

<script id="pyramid">
class pyramid extends CGRAobject {
      constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);


        var vertices = [

          -0.5, -0.5, -0.5, // 0
           0.5, -0.5, -0.5, // 1
           0.5, -0.5,  0.5, // 2
          -0.5, -0.5,  0.5, // 3

           0.0,  0.5,  0.0  // 4
        ];

        var indices = [
          // sides
          0, 1, 4, //Left side
          1, 2, 4,
          2, 3, 4,
          3, 0, 4,

          // base
          0, 1, 2,
          0, 2, 3
        ];

        var colors = [];
        for (let i = 0; i < vertices.length; i += 3) {//check x of every vertex

          //If x==-0.5 assign color1 otherwise assign color2  
          const t = (vertices[i] + 0.5);

          colors.push(
            color1[0]*(1-t)+color2[0]*t,
            color1[1]*(1-t)+color2[1]*t,
            color1[2]*(1-t)+color2[2]*t
          );
        }

        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
  }
    
  draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


In [ ]:
%%html

<script id="pyramidT">
class pyramidT extends CGRAobject {
      constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);


        var vertices = [

          -0.5, -0.5, -0.5, // 0
           0.5, -0.5, -0.5, // 1
           0.5, -0.5,  0.5, // 2
          -0.5, -0.5,  0.5, // 3

           0.0,  0.5,  0.0  // 4
        ];

        var indices = [
          // sides
          0, 1, 4, //Left side
          1, 2, 4,
          2, 3, 4,
          3, 0, 4,

          // base
          0, 1, 2,
          0, 2, 3
        ];

        var texcoords = [];
        var colors = [];
        for (let i = 0; i < vertices.length; i += 3) {//check x of every vertex
          const x = vertices[i];
          const y = vertices[i+1];
          const z = vertices[i+2];
                                                      
          //If x==-0.5 assign color1 otherwise assign color2  
          const t = (vertices[i] + 0.5);

          colors.push(
            color1[0]*(1-t)+color2[0]*t,
            color1[1]*(1-t)+color2[1]*t,
            color1[2]*(1-t)+color2[2]*t
          );
                                                      
          const u = x + 0.5;
          const v = z + 0.5;
          texcoords.push(u, v);
        }

        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        // texcoords
        this.texcoordbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texcoords), this.gl.STATIC_DRAW);  
          
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
  }
    
    setTexture(tex){
        this.texture = tex;
    }
    
  draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        // texcoords
        const texLoc = gl.getAttribLocation(shader.shaderProgram, "in_Tex_Coord");
        if (texLoc !== -1) {
            gl.bindBuffer(gl.ARRAY_BUFFER, this.texcoordbuffer);
            gl.vertexAttribPointer(texLoc, 2, gl.FLOAT, false, 0, 0);
            gl.enableVertexAttribArray(texLoc);
        }

        // texture
        if (this.texture) {
            gl.activeTexture(gl.TEXTURE0);
            gl.bindTexture(gl.TEXTURE_2D, this.texture.textureid);
            const samplerLoc = gl.getUniformLocation(shader.shaderProgram, "myTexture");
            if (samplerLoc !== -1) {
                gl.uniform1i(samplerLoc, 0);
            }
        }
      
        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


In [ ]:
%%html
<canvas id="myCanvas1" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myapp1 extends DEECapp{
    counter = 0;
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        /*var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);*/
        
        var fragsrc_tex = document.getElementById("my-fragment-shader-texture").text;
        var vertsrc_tex = document.getElementById("my-vertex-shader-texture").text;
        
        this.shaderprog_tex = new DEECshader(this.gl);
        this.shaderprog_tex.srcShaders(vertsrc_tex, fragsrc_tex);

        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        
        // Lets create cube object  
        this.cube1 = new cubeT(this.gl, [1,0,0], [0,0,1]);
        this.cube1.setShader(this.shaderprog_tex);
        
        this.cube2 = new cylinderT(this.gl, [1,0,0], [0,0,1]);
        this.cube2.setShader(this.shaderprog_tex);
        
        this.cube3 = new diskT(this.gl, [1,0,0], [0,0,1]);
        this.cube3.setShader(this.shaderprog_tex);

        this.cone = new coneT(this.gl, [1,0,0], [0,0,1]);
        this.cone.setShader(this.shaderprog_tex); 
        
        this.sphere = new sphereT(this.gl, [1,0,0], [0,0,1]);
        this.sphere.setShader(this.shaderprog_tex); 
        
        
        
        this.trans1 = glm.translate(glm.mat4(1.0),glm.vec3(4,6,0));
        this.trans2 = glm.translate(glm.mat4(1.0),glm.vec3(2,8,0));
        this.trans3 = glm.translate(glm.mat4(1.0),glm.vec3(0,6,0));
        this.trans4 = glm.translate(glm.mat4(1.0),glm.vec3(-2,8,0));
        this.trans5 = glm.translate(glm.mat4(1.0),glm.vec3(-4,6,0));

        this.projectionM = glm.perspective(glm.radians(45),1,0.1,1000);
        this.viewM = glm.lookAt(glm.vec3(0,10,12),glm.vec3(0,8,0),glm.vec3(0,1,0));

        // textures
        this.texture1 = new CGRAtexture(this.gl);
        this.texture1.load("Textures/train_metal.jpg");

        this.cube1.setTexture(this.texture1);
        
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("Textures/water.jpg");

        this.sphere.setTexture(this.texture2);
        
        this.texture3 = new CGRAtexture(this.gl);
        this.texture3.load("Textures/tree_leaf2.jpg");

        this.cone.setTexture(this.texture3);
        
        this.sphere.setTexture(this.texture2);
        
        this.texture4 = new CGRAtexture(this.gl);
        this.texture4.load("Textures/camel1.jpg");

        this.cube2.setTexture(this.texture4);
        
        this.texture5 = new CGRAtexture(this.gl);
        this.texture5.load("Textures/wood_bench.jpg");

        this.cube3.setTexture(this.texture5);
        
    }

    
    render(){
        
        this.shaderprog_tex.startUsing();
        
        this.counter++;
        
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        var MVPlocation = this.gl.getUniformLocation(this.shaderprog_tex.shaderProgram, "MVP");
        
        
        var rot = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(0.0,0.0,1.0)));
        this.model1 = this.trans1['*'](rot);
        this.cube1.setModelTransformation(this.model1);
        
        this.cube1.draw(this.viewM, this.projectionM, this.shaderprog_tex);
        
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(1.0,0.0,0.0)));
        this.model2 = this.trans2['*'](rot);
        this.cube2.setModelTransformation(this.model2);
    
        this.cube2.draw(this.viewM, this.projectionM, this.shaderprog_tex);
           
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(0.0,1.0,0.0)));
        this.model3 = this.trans3['*'](rot);
        this.cube3.setModelTransformation(this.model3);
        
        this.cube3.draw(this.viewM, this.projectionM, this.shaderprog_tex);

        
        rot = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(1.0,0.0,0.0)));
        this.model4 = this.trans4['*'](rot);
        this.cone.setModelTransformation(this.model4);
    
        this.cone.draw(this.viewM, this.projectionM, this.shaderprog_tex);
        
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(1.0,0.0,0.0)));
        this.model5 = this.trans5['*'](rot);
        this.sphere.setModelTransformation(this.model5);
    
        this.sphere.draw(this.viewM, this.projectionM, this.shaderprog_tex);

          
    }
}

var app1 = new myapp1('myCanvas1');

app1.run();

</script>

### Train

O comboio é um objeto composto que recebe as matrizes de rotação de cada carruagem, definida pelas distancias (a) e (b), tal como o espacamento entre cada carruagem (theta). De modo a calcular a orientação do comboio, foi calculada tangente das derivadas em relação à trajetória do comboio.

In [ ]:
%%html
<script id="locomotiveEngine">

class locomotiveEngine extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1], mode=0){
        super(glcontext);
        
        this.color1 = color1;
        this.color2 = color2;
        
        this.cabine = new cubeT(this.gl, color1, color2);
        this.lower_body = new cubeT(this.gl, color1, color2);

        this.engine = new cylinderT(this.gl, color1, color2);
        this.chimney = new cylinderT(this.gl, color1, color2);
        this.nose = new cylinderT(this.gl, color1, color2);
        this.cabine_roof = new cylinderT(this.gl, color1, color2);

        this.chimney_top = new coneT(this.gl, color1, color2);
        
        this.coolers = [];
        this.swheels = [];
        this.bwheels = [];
        
        this.mode = mode;
        
        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.cabine.setShader(shaderprog);
        this.lower_body.setShader(shaderprog);
        this.engine.setShader(shaderprog);
        this.chimney.setShader(shaderprog);
        this.nose.setShader(shaderprog);
        this.cabine_roof.setShader(shaderprog);
        this.chimney_top.setShader(shaderprog);
        
        for (let c of this.coolers) c.setShader(shaderprog);
        for (let w of this.swheels) w.setShader(shaderprog);
        for (let b of this.bwheels) b.setShader(shaderprog);
    }
    
    setTexture(bodyTex, wheelTex) {
        this.bodyTex  = bodyTex;
        this.wheelTex = wheelTex;

        this.cabine.setTexture(bodyTex);
        this.lower_body.setTexture(bodyTex);
        this.engine.setTexture(bodyTex);
        this.chimney.setTexture(bodyTex);
        this.nose.setTexture(bodyTex);
        this.cabine_roof.setTexture(bodyTex);
        this.chimney_top.setTexture(bodyTex);

        for (let c of this.coolers) {
            c.setTexture(bodyTex);
        }

        for (let w of this.swheels) {
            w.setTexture(wheelTex);
        }
        
        for (let b of this.bwheels) {
            b.setTexture(wheelTex);
        }
    }
    
    create_obj(){
        const c1 = this.color1;
        const c2 = this.color2;
        
        // lower_body
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(6.0, 2.0, 1.0));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        let transform = pos['*'](rot)['*'](scale);
        this.lower_body.setModelTransformation(transform);

        // cabine
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2.0, 1.5, 2.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.75, -2.0));
        transform = pos['*'](rot)['*'](scale);
        this.cabine.setModelTransformation(transform);

        // engine
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2.25, 4.0, 2.25));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.5, 1.0));
        transform = pos['*'](rot)['*'](scale);
        this.engine.setModelTransformation(transform);

        // chimney
        rot = glm.toMat4(glm.angleAxis(glm.radians(180.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.75, 1.0, 0.75));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 2.9, 2.5));
        transform = pos['*'](rot)['*'](scale);
        this.chimney.setModelTransformation(transform);

        // chimney top
        rot = glm.toMat4(glm.angleAxis(glm.radians(180.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.mat4(glm.mat3(0.5));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 3.2, 2.5));
        transform = pos['*'](rot)['*'](scale);
        this.chimney_top.setModelTransformation(transform);

        // small wheels
        const smallWheelPositions = [
            glm.vec3(0.7, -0.25, 2.5),   // right front
            glm.vec3(0.7, -0.25, 1.25),   // right back
            glm.vec3(-0.7, -0.25, 2.5),  // left front
            glm.vec3(-0.7, -0.25, 1.25)   // left back
        ];

        this.swheels = [];

        for (let i = 0; i < smallWheelPositions.length; i++) {
            const isRightSide = smallWheelPositions[i].x > 0;
        
            const color1 = isRightSide ? c1 : c2;
            const color2 = isRightSide ? c2 : c1;
            
            var wheel = new cylinderT(this.gl, color1, color2);
            
            if (this.mode==0){
                wheel = new cylinderT(this.gl, color1, color2);
            }else if (this.mode==1){
                wheel = new sphereT(this.gl, color1, color2);
            }else if (this.mode==2){
                wheel = new cubeT(this.gl, color1, color2);
            }
            
            const rot = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0, 0.0, 1.0)));
            const scale = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 0.5, 1.0));
            const pos = glm.translate(glm.mat4(1.0), smallWheelPositions[i]);
            const baseTransform = pos['*'](rot)['*'](scale);
            
            wheel.baseTransform = baseTransform;
            
            this.swheels.push(wheel);
        }

        // big wheel        
        const bigWheelPositions = [
            glm.vec3(-0.75, 0.15, -0.5),
            glm.vec3(-0.75, 0.15, -2.25),
            glm.vec3(0.75, 0.15, -0.5),
            glm.vec3(0.75, 0.15, -2.25)
        ];

        this.bwheels = [];

        for (let i = 0; i < bigWheelPositions.length; i++) {
            const isRightSide = bigWheelPositions[i].x > 0;
            
            const color1 = isRightSide ? c1 : c2;
            const color2 = isRightSide ? c2 : c1;
            
            var wheel = new cylinderT(this.gl, color1, color2);
            
            if (this.mode==0){
                wheel = new cylinderT(this.gl, color1, color2);
            }else if (this.mode==1){
                wheel = new sphereT(this.gl, color1, color2);
            }else if (this.mode==2){
                wheel = new cubeT(this.gl, color1, color2);
            }
            
            rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,1.0)));
            scale = glm.scale(glm.mat4(1.0), glm.vec3(1.75, 0.5, 1.75));
            const pos = glm.translate(glm.mat4(1.0), bigWheelPositions[i]);
            const baseTransform = pos['*'](rot)['*'](scale);
            
            wheel.baseTransform = baseTransform;
            
            this.bwheels.push(wheel);
        }

        // cabine roof
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.mat4(glm.mat3(2.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 2.3, -2.0));
        transform = pos['*'](rot)['*'](scale);
        this.cabine_roof.setModelTransformation(transform);

        // coolers
        const coolerPositions = [
            glm.vec3(0.6, 0.75, 1.75),
            glm.vec3(-0.6, 0.75, 1.75),
        ];

        this.coolers = [];

        for (let i = 0; i < coolerPositions.length; i++) {
            const cooler = new cylinderT(this.gl, c1, c2);
            rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
            scale = glm.scale(glm.mat4(1.0), glm.vec3(0.75, 2.5, 0.75));
            const pos = glm.translate(glm.mat4(1.0), coolerPositions[i]);
            const transform = pos['*'](rot)['*'](scale);
            cooler.setModelTransformation(transform);
            this.coolers.push(cooler);
        }

        // nose
        rot = glm.toMat4(glm.angleAxis(glm.radians(270.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.75, 0.25, 0.75));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.5, 3.0));
        transform = pos['*'](rot)['*'](scale);
        this.nose.setModelTransformation(transform);
    }

    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
        
        this.lower_body.draw(viewM, projectionM, this.shader, parentM);
        this.cabine.draw(viewM, projectionM, this.shader, parentM);
        this.engine.draw(viewM, projectionM, this.shader, parentM);
        this.chimney.draw(viewM, projectionM, this.shader, parentM);
        this.chimney_top.draw(viewM, projectionM, this.shader, parentM);
        this.nose.draw(viewM, projectionM, this.shader, parentM);
        this.cabine_roof.draw(viewM, projectionM, this.shader, parentM);
        
        for (let c of this.coolers) {
            c.draw(viewM, projectionM, this.shader, parentM);
        }
        
        for (let w of this.swheels) {
            const angle = (counter % (2 * Math.PI)) * 2;
            const spin = glm.toMat4(glm.angleAxis(angle, glm.vec3(0.0,1.0,0.0)));
            w.setModelTransformation(w.baseTransform['*'](spin));
            w.draw(viewM, projectionM, this.shader, parentM);
        }
        
        for (let b of this.bwheels) {
            const angle = (counter % (2 * Math.PI)) * 2;
            const spin = glm.toMat4(glm.angleAxis(angle, glm.vec3(0.0,1.0,0.0)));
            b.setModelTransformation(b.baseTransform['*'](spin));
            b.draw(viewM, projectionM, this.shader, parentM);
        }
    }
}


</script>


In [ ]:
%%html
<script id="locomotiveCarriage">

class locomotiveCarriage extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1], carriageSize = 2, mode=0){
        super(glcontext);
        
        this.carriageSize = carriageSize; // 1, 2, 3
        this.color1 = color1;
        this.color2 = color2;
        
        this.body = new cubeT(this.gl, color1, color2);
        this.lower_body = new cubeT(this.gl, color1, color2);

        this.roof = new cylinderT(this.gl, color1, color2);

        this.swheels = [];
        
        this.mode = mode;
        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.body.setShader(shaderprog);
        this.lower_body.setShader(shaderprog);
        this.roof.setShader(shaderprog);
        
        for (let w of this.swheels) w.setShader(shaderprog);
    }
    
    setTexture(bodyTex, wheelTex, cgraTex = null) {
        this.bodyTex  = bodyTex;
        this.wheelTex = wheelTex;
        this.cgraTex = cgraTex;

        let useBodyTex = bodyTex;
        if (cgraTex) {
            useBodyTex = cgraTex;
        }
        this.body.setTexture(useBodyTex);
        
        this.lower_body.setTexture(bodyTex);
        this.roof.setTexture(bodyTex);

        for (let w of this.swheels) {
            w.setTexture(wheelTex);
        }
    }
    
    create_obj(){
        const lengthMap = {1: 3.0, 2: 6.0, 3: 9.0};
        const bodyLength = lengthMap[this.carriageSize] || 6.0;
        
        const c1 = this.color1;
        const c2 = this.color2;
        
        // lower_body
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(bodyLength, 1.0, 1.0));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        let transform = pos['*'](rot)['*'](scale);
        this.lower_body.setModelTransformation(transform);

        // body
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(bodyLength, 2.5, 2.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.body.setModelTransformation(transform);

        
        this.swheels = [];
        let wheelZPositions;
        if(bodyLength === 3.0) wheelZPositions = [1.0, -1.0];
        else if(bodyLength === 6.0) wheelZPositions = [2.5, 1.25, -1.25, -2.5];
        else wheelZPositions = [3.75, 2.5, 1.25, -1.25, -2.5, -3.75];

        for(let z of wheelZPositions){
            for(let side of [-1,1]){ // left/right
                const color1 = side > 0 ? c1 : c2;
                const color2 = side > 0 ? c2 : c1;
                                    
                var wheel = new cylinderT(this.gl, color1, color2);
                                    
                if (this.mode==0){
                    wheel = new cylinderT(this.gl, color1, color2);
                } else if (this.mode==1){
                    wheel = new sphereT(this.gl, color1, color2);
                } else if (this.mode==2){
                    wheel = new cubeT(this.gl, color1, color2);
                }

                const rotW = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0, 0.0, 1.0)));
                const scaleW = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 0.5, 1.0));
                const posW = glm.translate(glm.mat4(1.0), glm.vec3(0.7*side, -0.25, z));
                wheel.baseTransform = posW['*'](rotW)['*'](scaleW);

                this.swheels.push(wheel);
            }
        }

        // roof
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2.0, bodyLength, 2.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 2.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.roof.setModelTransformation(transform);

    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
        
        this.lower_body.draw(viewM, projectionM, this.shader, parentM);
        this.body.draw(viewM, projectionM, this.shader, parentM);
        this.roof.draw(viewM, projectionM, this.shader, parentM);
        
        for (let w of this.swheels) {
            const angle = (counter % (2 * Math.PI)) * 2;
            const spin = glm.toMat4(glm.angleAxis(angle, glm.vec3(0.0,1.0,0.0)));
            w.setModelTransformation(w.baseTransform['*'](spin));
            w.draw(viewM, projectionM, this.shader, parentM);
        }
        
    }
}


</script>

In [ ]:
%%html
<script id="ellipsePose">

    function ellipsePose(a, b, alpha, t, eps) {
      // Parameter along ellipse
      const th = alpha * t + eps;

      // Position
      const x =  a * Math.cos(th);
      const z =  b * Math.sin(th);

      // Tangent
      let dx = -a * alpha * Math.sin(th);
      let dz =  b * alpha * Math.cos(th);

      const T = glm.translate(glm.mat4(1.0), glm.vec3(x, 0.75, z));
      const R = glm.toMat4(glm.angleAxis(Math.atan2(dx, dz), glm.vec3(0.0, 1.0, 0.0)));
      return T['*'](R);
    }

</script>

In [ ]:
%%html
<script id="train">

class train extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1], mode=0){
        super(glcontext);
        
        this.head = new locomotiveEngine(this.gl, color1, color2,mode);
        this.c1 = new locomotiveCarriage(this.gl, color1, color2, 2,mode);
        this.c2 = new locomotiveCarriage(this.gl, color1, color2, 1,mode);
        this.c3 = new locomotiveCarriage(this.gl, color1, color2, 3,mode);

        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.head.setShader(shaderprog);
        this.c1.setShader(shaderprog);
        this.c2.setShader(shaderprog);
        this.c3.setShader(shaderprog);
    }
    
    setTexture(engineTex, bodyTex, wheelTex, cgraTex = null) {
        this.engineTex = engineTex;
        this.bodyTex  = bodyTex;
        this.wheelTex = wheelTex;
        this.cgraTex = cgraTex;

        this.head.setTexture(engineTex, wheelTex);
        this.c1.setTexture(bodyTex, wheelTex, cgraTex);
        this.c2.setTexture(bodyTex, wheelTex);
        this.c3.setTexture(bodyTex, wheelTex);
    }
    
    create_obj(){
       const I = glm.mat4(1.0);
       this.head.setModelTransformation(I);
       this.c1.setModelTransformation(I);
       this.c2.setModelTransformation(I);
       this.c3.setModelTransformation(I);
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), M0 = glm.mat4(1.0), M1 = glm.mat4(1.0), M2 = glm.mat4(1.0), M3 = glm.mat4(1.0), counter = 0){
          const base = parentM['*'](this.modelMat);
          this.head.draw(viewM, projectionM, base['*'](M0), counter);
          this.c1.draw(viewM,  projectionM, base['*'](M1), counter);
          this.c2.draw(viewM,  projectionM, base['*'](M2), counter);
          this.c3.draw(viewM,  projectionM, base['*'](M3), counter);
    }
}


</script>

In [ ]:
%%html
<canvas id="myCanvas4" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp4">

class myapp4 extends DEECapp{    
    counter = 0;
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader-texture").text;
        var vertsrc = document.getElementById("my-vertex-shader-texture").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.train = new train(this.gl, [0.0, 0.75, 1.0], [1.0, 0.0, 0.25]);
        this.train.setShader(this.shaderprog);
        
        this.a = 20.0;     // ellipse radius X
        this.b = 12.0;     // ellipse radius Z
        this.alpha = 0.4;  // angular speed
        
        this.projectionM = glm.perspective(glm.radians(45), 1.0, 0.1, 1000.0);
        this.viewM = glm.lookAt(glm.vec3(30, 30, 30), glm.vec3(0, 0, 0), glm.vec3(0, 1, 0));
        
        // Textures
        this.engineTexture = new CGRAtexture(this.gl);
        this.engineTexture.load("Textures/locomotive_metal2.jpg");
        
        this.bodyTexture = new CGRAtexture(this.gl);
        this.bodyTexture.load("Textures/train_metal.jpg");

        this.cgraTexture = new CGRAtexture(this.gl);
        this.cgraTexture.load("Textures/cgra_texture2.jpg");
        
        this.wheelTexture = new CGRAtexture(this.gl);
        this.wheelTexture.load("Textures/locomotive_metal2.jpg");
        
        this.train.setTexture(this.engineTexture, this.bodyTexture, this.wheelTexture, this.cgraTexture);
        
        this.start = Date.now();
    }
    
    
    
    render(){
        var t = (this.start-Date.now())/1000.0;
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        //Train
        
        const I = glm.mat4(1.0);

        const M0 = ellipsePose(this.a, this.b, this.alpha, -t, -Math.PI/2);
        const M1 = ellipsePose(this.a, this.b, this.alpha, -t,  -2.3*Math.PI/4);
        const M2 = ellipsePose(this.a, this.b, this.alpha, -t,  -2.6*Math.PI/4);
        const M3 = ellipsePose(this.a, this.b, this.alpha, -t,  -3*Math.PI/4);

        this.train.draw(this.viewM, this.projectionM, I, M0, M1, M2, M3, -t);
        }

}


var app4 = new myapp4('myCanvas4');

app4.run();

</script>

In [ ]:
%%html
<script id="lampPost">

class lampPost extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.base1 = new cylinderT(this.gl, color1, color2);
        this.base2 = new coneT(this.gl, color1, color2);
        this.body = new cylinderT(this.gl, color1, color2);
        
        this.top1 = new sphereT(this.gl, color1, color2);
        this.top2 = new coneT(this.gl, color1, color2);
        
        this.lamp = new coneT(this.gl, [1.0, 0.894, 0.549], [0.790, 0.553, 0.0790]);
        
        this.disk = new diskT(this.gl, color1, color2);
        
        this.create_obj();
    }
    
    setShader(shaderprog){       
        this.shader = shaderprog;
        
        this.base1.setShader(shaderprog);
        this.base2.setShader(shaderprog);
        this.body.setShader(shaderprog);
        this.top1.setShader(shaderprog);
        this.top2.setShader(shaderprog);
        this.lamp.setShader(shaderprog);
        this.disk.setShader(shaderprog);
    }
    
    setTexture(metalTex, lampTex) {
        this.metalTex = metalTex;
        this.lampTex  = lampTex;

        // partes metálicas
        this.base1.setTexture(metalTex);
        this.base2.setTexture(metalTex);
        this.body.setTexture(metalTex);
        this.top1.setTexture(metalTex);
        this.top2.setTexture(metalTex);
        this.disk.setTexture(metalTex);

        // lâmpada
        this.lamp.setTexture(lampTex);
    }
    
    create_obj(){
        
        // base1
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,0.0)));
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 0.25, 1.0));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, 0.0));
        let transform = pos['*'](rot)['*'](scale);
        this.base1.setModelTransformation(transform);
        
        // base2
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.7, 0.8, 0.7));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.base2.setModelTransformation(transform);
        
        // body
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.35, 5.0, 0.35));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 3.0, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.body.setModelTransformation(transform);
        
        // lamp
        rot = glm.toMat4(glm.angleAxis(glm.radians(180.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1, 1.2, 1));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 5.0, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.lamp.setModelTransformation(transform);
        
        // disk
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1.5, 1.5, 1.5));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 5.75, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.disk.setModelTransformation(transform);
        
        // top1
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.75, 0.75, 0.75));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 5.75, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.top1.setModelTransformation(transform);
        
        // top2
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.5, 1, 0.5));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 6.0, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.top2.setModelTransformation(transform);
        
        
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
          const base = parentM['*'](this.modelMat);
        
          this.base1.draw(viewM, projectionM, this.shader, base);
          this.base2.draw(viewM, projectionM, this.shader, base);
          this.body.draw(viewM, projectionM, this.shader, base);
          this.lamp.draw(viewM, projectionM, this.shader, base);
          this.disk.draw(viewM, projectionM, this.shader, base);
          this.top1.draw(viewM, projectionM, this.shader, base);
          this.top2.draw(viewM, projectionM, this.shader, base);
     }

}

</script>

In [ ]:
%%html
<script id="camelNeck">

class camelNeck extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.color1 = color1;
        this.color2 = color2;
        
        this.head1 = new cylinderT(this.gl, color1, color2);
        this.neck1 = new cylinderT(this.gl, color2, color1);
        this.neck2 = new cylinderT(this.gl, color2, color1);
        
        this.ears = [];
        
        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.head1.setShader(shaderprog);
        this.neck1.setShader(shaderprog);
        this.neck2.setShader(shaderprog);
        
        for (let e of this.ears) e.setShader(shaderprog);
    }
    
    setTexture(camelTex) {
        this.camelTex = camelTex;

        this.head1.setTexture(camelTex);
        this.neck1.setTexture(camelTex);
        this.neck2.setTexture(camelTex);

        for (let e of this.ears) {
            e.setTexture(this.camelTex);
        }
    }
    
    create_obj(){
        const c1 = this.color1;
        const c2 = this.color2;
        
        // head1
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0,0.0,1.0)));
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(0.75, 0.5, 1.25));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 2.2, 3.3));
        let transform = pos['*'](rot)['*'](scale);
        this.head1.setModelTransformation(transform);
        
        // neck1
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.9,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.75, 2.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.25, 1.6));
        transform = pos['*'](rot)['*'](scale);
        this.neck1.setModelTransformation(transform);
        
        // neck2
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.6, 0.0, 0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.7, 1.25, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.7, 2.7));
        transform = pos['*'](rot)['*'](scale);
        this.neck2.setModelTransformation(transform);
        
        // ears
        const earPositions = [
            glm.vec3(-0.3, 2.4, 3.0),   // right
            glm.vec3(0.3, 2.4, 3.0)   // left
        ];
        
        const earRotations = [
            glm.radians(90.0),
            glm.radians(270.0)
        ];

        this.ears = [];

        for (let i = 0; i < earPositions.length; i++) {
        
            const ear = new cylinderT(this.gl, c1, c2);
            
            const rot = glm.toMat4(glm.angleAxis(earRotations[i], glm.vec3(0.0, 0.0, 0.75)));
            const scale = glm.scale(glm.mat4(1.0), glm.vec3(0.2, 0.4, 0.2));
            const pos = glm.translate(glm.mat4(1.0), earPositions[i]);
            const baseTransform = pos['*'](rot)['*'](scale);
            
            ear.setModelTransformation(baseTransform);
            this.ears.push(ear);
        }
        
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
          const base = parentM['*'](this.modelMat);
          
          this.head1.draw(viewM, projectionM, this.shader, base);

          this.neck1.draw(viewM, projectionM, this.shader, base);
          this.neck2.draw(viewM, projectionM, this.shader, base);
        
          for (let e of this.ears) e.draw(viewM, projectionM, this.shader, base);
     }

}


</script>

In [ ]:
%%html
<script id="camel">

class camel extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.color1 = color1;
        this.color2 = color2;
        
        this.body1 = new cylinderT(this.gl, color2, color1);
        this.tail = new cylinderT(this.gl, color2, color1);
        
        this.butt = new sphereT(this.gl, color1, color2);
        
        this.legs = [];
        this.humps = [];
        
        this.neck = new camelNeck(this.gl, color1, color2);
        
        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.body1.setShader(shaderprog);
        this.tail.setShader(shaderprog);
        
        this.butt.setShader(shaderprog);
        
        for (let l of this.legs) l.setShader(shaderprog);
        for (let h of this.humps) h.setShader(shaderprog);
        
        this.neck.setShader(shaderprog);
    }
    
    setTexture(camelTex) {
        this.camelTex = camelTex;

        this.body1.setTexture(camelTex);
        this.tail.setTexture(camelTex);
        this.butt.setTexture(camelTex);

        for (let l of this.legs)  l.setTexture(camelTex);
        for (let h of this.humps) h.setTexture(camelTex);

        this.neck.setTexture(camelTex);
    }
    
    create_obj(){
        const c1 = this.color1;
        const c2 = this.color2;
        
        // body1
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(1.8, 3.5, 2.3));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.25, 0.0));
        let transform = pos['*'](rot)['*'](scale);
        this.body1.setModelTransformation(transform);
        
        // tail
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.4,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.25, 1.5, 0.25));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.3, -2.3));
        transform = pos['*'](rot)['*'](scale);
        this.tail.setModelTransformation(transform);
        
        // butt
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0, 0.0, 0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1.6, 2.4, 2.4));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.25, -1.25));
        transform = pos['*'](rot)['*'](scale);
        this.butt.setModelTransformation(transform);
        
        // legs
        const legPositions = [
            glm.vec3(0.6, 0.0, 1.5),   
            glm.vec3(0.6, 0.0, -1.5),
            glm.vec3(-0.6, 0.0, 1.5),
            glm.vec3(-0.6, 0.0, -1.5)
        ];

        this.legs = [];

        for (let i = 0; i < legPositions.length; i++) {
        
            const leg = new cylinderT(this.gl, c1, c2);
            
            const rot = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0, 0.0, 0.0)));
            const scale = glm.scale(glm.mat4(1.0), glm.vec3(0.5, 3.0, 0.5));
            const pos = glm.translate(glm.mat4(1.0), legPositions[i]);
            const baseTransform = pos['*'](rot)['*'](scale);
            
            leg.setModelTransformation(baseTransform);
            this.legs.push(leg);
        }
        
        // humps
        const humpPositions = [
            glm.vec3(0.0, 1.9, -1.0),
            glm.vec3(0.0, 2.0, 0.75)
        ];
        
        const humpScales = [
            glm.vec3(2.5, 1.2, 1.5),
            glm.vec3(2.5, 1.2, 2)
        ];

        this.humps = [];

        for (let i = 0; i < humpPositions.length; i++) {
        
            const hump = new sphereT(this.gl, c2, c1);
            
            const rot = glm.toMat4(glm.angleAxis(glm.radians(270.0), glm.vec3(0.0, 0.0, 1.0)));
            const scale = glm.scale(glm.mat4(1.0), humpScales[i]);
            const pos = glm.translate(glm.mat4(1.0), humpPositions[i]);
            const baseTransform = pos['*'](rot)['*'](scale);
            
            hump.setModelTransformation(baseTransform);
            this.humps.push(hump);
        }
        
        // neck
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0, 0.0, 0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 1.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.neck.setModelTransformation(transform);
              
    }
    
    setNeckAngle(angle){
      let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(angle, 0.0, 0.0)));
      let scale = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 1.0, 1.0));
      let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, 0.0));
      let transform = pos['*'](rot)['*'](scale);
      this.neck.setModelTransformation(transform);
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
          const base = parentM;
          
          const t = counter/5; //~30 second loop  (5 times slower than a normal loop, 6.28s)
          const angle = 0.3 * (Math.sin(t) + 1.0);  // normalize between [0.3, 0.6] to limit neck position

          this.setNeckAngle(angle);
        
          this.body1.draw(viewM, projectionM, this.shader, base);

          this.tail.draw(viewM, projectionM, this.shader, base);
        
          this.butt.draw(viewM, projectionM, this.shader, base);

          for (let l of this.legs) l.draw(viewM, projectionM, this.shader, base);
          for (let h of this.humps) h.draw(viewM, projectionM, this.shader, base);
        
          this.neck.draw(viewM, projectionM, base, counter = 0);
     }

}


</script>

In [1]:
%%html
<script id="sphynx">

class sphynx extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.color1 = color1;
        this.color2 = color2;
        
        
        this.floor = new cubeT(this.gl, color1, color2);
        this.ceiling = new cubeT(this.gl, color1, color2);
        this.ceiling2 = new cubeT(this.gl, color1, color2);
        this.ceiling3 = new cubeT(this.gl, color1, color2);
        this.ceiling4 = new cubeT(this.gl, color1, color2);
        
        this.backwall = new cubeT(this.gl, color1, color2);
        
        this.leftwall = new cubeT(this.gl, color1, color2);
        this.leftwall1 = new cubeT(this.gl, color1, color2);
        this.leftwall2 = new cubeT(this.gl, color1, color2);
        this.leftwall3 = new cubeT(this.gl, color1, color2);
        this.rightwall = new cubeT(this.gl, color1, color2);
        this.rightwall1 = new cubeT(this.gl, color1, color2);
        this.rightwall2 = new cubeT(this.gl, color1, color2);
        this.rightwall3 = new cubeT(this.gl, color1, color2);
        
        
        this.leftpaw_back = new cylinderT(this.gl, color1, color2);
        this.rightpaw_back = new cylinderT(this.gl, color1, color2);
        
        this.leftpaw_front1 = new cylinderT(this.gl, color1, color2);
        this.leftpaw_front2 = new cylinderT(this.gl, color1, color2);
        this.leftpaw_front3 = new cylinderT(this.gl, color1, color2);
        this.leftpaw_front4 = new cylinderT(this.gl, color1, color2);
        
        this.rightpaw_front1 = new cylinderT(this.gl, color1, color2);
        this.rightpaw_front2 = new cylinderT(this.gl, color1, color2);
        this.rightpaw_front3 = new cylinderT(this.gl, color1, color2);
        this.rightpaw_front4 = new cylinderT(this.gl, color1, color2);
        
        
        this.leftpaw_back_sphere = new sphereT(this.gl, color1, color2);
        this.rightpaw_back_sphere = new sphereT(this.gl, color1, color2);
        
        this.leftpaw_back_sphere2 = new sphereT(this.gl, color1, color2);
        this.rightpaw_back_sphere2 = new sphereT(this.gl, color1, color2);
        
        this.leftpaw_front1_sphere2 = new sphereT(this.gl, color1, color2);
        this.rightpaw_front1_sphere2 = new sphereT(this.gl, color1, color2);
        
        this.leftpaw_front1_sphere = new sphereT(this.gl, color1, color2);
        this.leftpaw_front2_sphere = new sphereT(this.gl, color1, color2);
        this.leftpaw_front3_sphere = new sphereT(this.gl, color1, color2);
        this.leftpaw_front4_sphere = new sphereT(this.gl, color1, color2);
        
        this.rightpaw_front1_sphere = new sphereT(this.gl, color1, color2);
        this.rightpaw_front2_sphere = new sphereT(this.gl, color1, color2);
        this.rightpaw_front3_sphere = new sphereT(this.gl, color1, color2);
        this.rightpaw_front4_sphere = new sphereT(this.gl, color1, color2);
        
        this.frontwall_top = new cubeT(this.gl, color1, color2);
        this.frontwall_left = new cubeT(this.gl, color1, color2);
        this.frontwall_right = new cubeT(this.gl, color1, color2);
        
        
        this.neck = new cylinderT(this.gl, color1, color2);
        this.head = new cylinderT(this.gl, color1, color2);
        
        this.cone1 = new coneT(this.gl, color1, color2)
        this.cone2 = new coneT(this.gl, color1, color2)
        this.cone3 = new coneT(this.gl, color1, color2)
        this.cone4 = new coneT(this.gl, color1, color2)
        this.cone5 = new coneT(this.gl, color1, color2)
        this.cone6 = new coneT(this.gl, color1, color2)
        this.cone7 = new coneT(this.gl, color1, color2)
        this.cone8 = new coneT(this.gl, color1, color2)
        
        this.face = new sphereT(this.gl, color1, color2)
        
        
        this.ceiling_border_1 = new cylinderT(this.gl, color1, color2);
        this.ceiling_border_2 = new cylinderT(this.gl, color1, color2);
        this.ceiling_border_3 = new cylinderT(this.gl, color1, color2);
        this.ceiling_border_4 = new cylinderT(this.gl, color1, color2);
        
        this.ceiling_border_5 = new cylinderT(this.gl, color1, color2);
        this.ceiling_border_6 = new cylinderT(this.gl, color1, color2);
        this.ceiling_border_7 = new cylinderT(this.gl, color1, color2);
        this.ceiling_border_8 = new cylinderT(this.gl, color1, color2);
        
        this.ceiling_border_9 = new cylinderT(this.gl, color1, color2);
        this.ceiling_border_10 = new cylinderT(this.gl, color1, color2);
        this.ceiling_border_11 = new cylinderT(this.gl, color1, color2);
        this.ceiling_border_12 = new cylinderT(this.gl, color1, color2);
        
        this.tail = new cylinderT(this.gl, color1, color2);

        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.floor.setShader(shaderprog);
        this.ceiling.setShader(shaderprog);
        this.ceiling2.setShader(shaderprog);
        this.ceiling3.setShader(shaderprog);
        this.ceiling4.setShader(shaderprog);
        this.backwall.setShader(shaderprog);

        this.leftwall.setShader(shaderprog);
        this.leftwall1.setShader(shaderprog);
        this.leftwall2.setShader(shaderprog);
        this.leftwall3.setShader(shaderprog);
        this.rightwall.setShader(shaderprog);
        this.rightwall1.setShader(shaderprog);
        this.rightwall2.setShader(shaderprog);
        this.rightwall3.setShader(shaderprog);

        this.frontwall_top.setShader(shaderprog);
        this.frontwall_left.setShader(shaderprog);
        this.frontwall_right.setShader(shaderprog);

        this.leftpaw_back.setShader(shaderprog);
        this.rightpaw_back.setShader(shaderprog);

        this.leftpaw_front1.setShader(shaderprog);
        this.leftpaw_front2.setShader(shaderprog);
        this.leftpaw_front3.setShader(shaderprog);
        this.leftpaw_front4.setShader(shaderprog);

        this.rightpaw_front1.setShader(shaderprog);
        this.rightpaw_front2.setShader(shaderprog);
        this.rightpaw_front3.setShader(shaderprog);
        this.rightpaw_front4.setShader(shaderprog);
        
        this.leftpaw_back_sphere.setShader(shaderprog);
        this.rightpaw_back_sphere.setShader(shaderprog);
        
        this.leftpaw_back_sphere2.setShader(shaderprog);
        this.rightpaw_back_sphere2.setShader(shaderprog);
        
        this.leftpaw_front1_sphere2.setShader(shaderprog);
        this.rightpaw_front1_sphere2.setShader(shaderprog);

        this.leftpaw_front1_sphere.setShader(shaderprog);
        this.leftpaw_front2_sphere.setShader(shaderprog);
        this.leftpaw_front3_sphere.setShader(shaderprog);
        this.leftpaw_front4_sphere.setShader(shaderprog);

        this.rightpaw_front1_sphere.setShader(shaderprog);
        this.rightpaw_front2_sphere.setShader(shaderprog);
        this.rightpaw_front3_sphere.setShader(shaderprog);
        this.rightpaw_front4_sphere.setShader(shaderprog);
        
        this.neck.setShader(shaderprog);
        this.head.setShader(shaderprog);
        
        this.cone1.setShader(shaderprog);
        this.cone2.setShader(shaderprog);
        this.cone3.setShader(shaderprog);
        this.cone4.setShader(shaderprog);
        this.cone5.setShader(shaderprog);
        this.cone6.setShader(shaderprog);
        this.cone7.setShader(shaderprog);
        this.cone8.setShader(shaderprog);
        
        this.face.setShader(shaderprog);
        
         this.ceiling_border_1.setShader(shaderprog);
        this.ceiling_border_2.setShader(shaderprog);
        this.ceiling_border_3.setShader(shaderprog);
        this.ceiling_border_4.setShader(shaderprog);
        this.ceiling_border_5.setShader(shaderprog);
        this.ceiling_border_6.setShader(shaderprog);
        this.ceiling_border_7.setShader(shaderprog);
        this.ceiling_border_8.setShader(shaderprog);
        this.ceiling_border_9.setShader(shaderprog);
        this.ceiling_border_10.setShader(shaderprog);
        this.ceiling_border_11.setShader(shaderprog);
        this.ceiling_border_12.setShader(shaderprog);
        this.tail.setShader(shaderprog);
        
    }
    
    setTexture(sphynxTex, floorTex) {
        this.sphynxTex = sphynxTex;
        this.floorTex = floorTex;

        this.floor.setTexture(floorTex);
        
        
        this.ceiling.setTexture(sphynxTex);
        this.ceiling2.setTexture(sphynxTex);
        this.ceiling3.setTexture(sphynxTex);
        this.ceiling4.setTexture(sphynxTex);
        
        this.backwall.setTexture(sphynxTex);
        
        this.leftwall.setTexture(sphynxTex);
        this.leftwall1.setTexture(sphynxTex);
        this.leftwall2.setTexture(sphynxTex);
        this.leftwall3.setTexture(sphynxTex);
        this.rightwall.setTexture(sphynxTex);
        this.rightwall1.setTexture(sphynxTex);
        this.rightwall2.setTexture(sphynxTex);
        this.rightwall3.setTexture(sphynxTex);
        
        this.leftpaw_back.setTexture(sphynxTex);
        this.rightpaw_back.setTexture(sphynxTex);
        
        this.leftpaw_front1.setTexture(sphynxTex);
        this.leftpaw_front2.setTexture(sphynxTex);
        this.leftpaw_front3.setTexture(sphynxTex);
        this.leftpaw_front4.setTexture(sphynxTex);
        
        this.rightpaw_front1.setTexture(sphynxTex);
        this.rightpaw_front2.setTexture(sphynxTex);
        this.rightpaw_front3.setTexture(sphynxTex);
        this.rightpaw_front4.setTexture(sphynxTex);
        
        
        this.leftpaw_back_sphere.setTexture(sphynxTex);
        this.rightpaw_back_sphere.setTexture(sphynxTex);
        
        this.leftpaw_back_sphere2.setTexture(sphynxTex);
        this.rightpaw_back_sphere2.setTexture(sphynxTex);
        
        this.leftpaw_front1_sphere2.setTexture(sphynxTex);
        this.rightpaw_front1_sphere2.setTexture(sphynxTex);
        
        this.leftpaw_front1_sphere.setTexture(sphynxTex);
        this.leftpaw_front2_sphere.setTexture(sphynxTex);
        this.leftpaw_front3_sphere.setTexture(sphynxTex);
        this.leftpaw_front4_sphere.setTexture(sphynxTex);
        
        this.rightpaw_front1_sphere.setTexture(sphynxTex);
        this.rightpaw_front2_sphere.setTexture(sphynxTex);
        this.rightpaw_front3_sphere.setTexture(sphynxTex);
        this.rightpaw_front4_sphere.setTexture(sphynxTex);
        
        this.frontwall_top.setTexture(sphynxTex);
        this.frontwall_left.setTexture(sphynxTex);
        this.frontwall_right.setTexture(sphynxTex);
        
        
        this.neck.setTexture(sphynxTex);
        this.head .setTexture(sphynxTex);
        
        this.cone1.setTexture(sphynxTex);
        this.cone2.setTexture(sphynxTex);
        this.cone3.setTexture(sphynxTex);
        this.cone4.setTexture(sphynxTex);
        this.cone5.setTexture(sphynxTex);
        this.cone6.setTexture(sphynxTex);
        this.cone7.setTexture(sphynxTex);
        this.cone8.setTexture(sphynxTex);
        
        this.face.setTexture(sphynxTex);
    }
    
    create_obj(){
        
        const c1 = this.color1;
        const c2 = this.color2;
        
        //Floor
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(20.8, 43.9, 0.05));
        let rot = glm.toMat4(glm.angleAxis(glm.radians(-90.0), glm.vec3(1.0, 0.0, 0.0)));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0, -8.0));
        let transform = pos['*'](rot['*'](scale));
        this.floor.setModelTransformation(transform);
        
        //Ceiling        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(21, 1, 60));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 4.5 , 0.0));
        transform = pos['*'](scale);
        this.ceiling.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(19, 1, 55.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 5.5 , 0.0));
        transform = pos['*'](scale);
        this.ceiling2.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(17, 1, 50.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 6.5 , 0.0));
        transform = pos['*'](scale);
        this.ceiling3.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(15, 1, 45.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 7.5 , 0.0));
        transform = pos['*'](scale);
        this.ceiling4.setModelTransformation(transform);
        
        //Backwall
        scale = glm.scale(glm.mat4(1.0), glm.vec3(21, 4.4, 8.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 2.2, 26.0));
        transform = pos['*'](scale);
        this.backwall.setModelTransformation(transform);

        //Leftwall 
        scale = glm.scale(glm.mat4(1.0), glm.vec3(43.9, 1.0, 0.98));
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0,1.0,0.0)));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(10, 0.5, -8.0));
        transform = pos['*'](rot)['*'](scale);
        this.leftwall.setModelTransformation(transform);
        
        //Leftwall 1
        scale = glm.scale(glm.mat4(1.0), glm.vec3(43.9, 1.4, 0.98));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(10, 3.7, -8.0));
        transform = pos['*'](rot)['*'](scale);
        this.leftwall1.setModelTransformation(transform);
        
        //Leftwall 2
        scale = glm.scale(glm.mat4(1.0), glm.vec3(10, 4.4, 1));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(10, 2.2, 9.0));
        transform = pos['*'](rot)['*'](scale);
        this.leftwall2.setModelTransformation(transform);
        
        //Leftwall 3
        scale = glm.scale(glm.mat4(1.0), glm.vec3(10, 4.4, 1));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(10, 2.2, -24.8));
        transform = pos['*'](rot)['*'](scale);
        this.leftwall3.setModelTransformation(transform);
        
        //Rightwall 
        scale = glm.scale(glm.mat4(1.0), glm.vec3(43.9, 1, 0.98));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-10, 0.5, -8.0));
        transform = pos['*'](rot)['*'](scale);
        this.rightwall.setModelTransformation(transform);
        
        //Rightwall 1
        scale = glm.scale(glm.mat4(1.0), glm.vec3(43.9, 1.4, 0.98));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-10, 3.7, -8.0));
        transform = pos['*'](rot)['*'](scale);
        this.rightwall1.setModelTransformation(transform);
        
        //Rightwall 2
        scale = glm.scale(glm.mat4(1.0), glm.vec3(10, 4.4, 1));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-10, 2.2, 9.0));
        transform = pos['*'](rot)['*'](scale);
        this.rightwall2.setModelTransformation(transform);
        
        //Rightwall 3
        scale = glm.scale(glm.mat4(1.0), glm.vec3(10, 4.4, 1));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-10, 2.2, -24.8));
        transform = pos['*'](rot)['*'](scale);
        this.rightwall3.setModelTransformation(transform);

        //Frontwall top
        scale = glm.scale(glm.mat4(1.0), glm.vec3(21, 2.4, 0.2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 3.2, -29.9));
        transform = pos['*'](scale);
        this.frontwall_top.setModelTransformation(transform);
        
        //Frontwall left
        scale = glm.scale(glm.mat4(1.0), glm.vec3(8.5, 2.0, 0.2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-6.250, 1, -29.9));
        transform = pos['*'](scale);
        this.frontwall_left.setModelTransformation(transform);
        
        //Frontwall right
        scale = glm.scale(glm.mat4(1.0), glm.vec3(8.5, 2.0, 0.2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(6.250, 1, -29.9));
        transform = pos['*'](scale);
        this.frontwall_right.setModelTransformation(transform);
        
        //leftpaw back
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 6.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-11.55, 0, 25.0));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_back.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-11.55, 0, 22.0));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_back_sphere.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-11.55, 0, 28.0));
        transform = pos['*'](scale);
        this.leftpaw_back_sphere2.setModelTransformation(transform);
        
        
        //rightpaw back
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 6.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(11.55, 0, 25.0));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_back.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(11.55, 0, 22.0));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_back_sphere.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(11.55, 0, 28.0));
        transform = pos['*'](scale);
        this.rightpaw_back_sphere2.setModelTransformation(transform);
        
        
        //Leftpaw front 1
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 18.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-11.3, 0, -36.0));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_front1.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-11.3, 0, -27.0));
        transform = pos['*'](scale);
        this.leftpaw_front1_sphere2.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-11.3, 0, -45.0));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_front1_sphere.setModelTransformation(transform);
        
        
        
        //Rightpaw front 1
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 18.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(11.3, 0, -36.0));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_front1.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(11.3, 0, -45.0));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_front1_sphere.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(11.3, 0, -27.0));
        transform = pos['*'](scale);
        this.rightpaw_front1_sphere2.setModelTransformation(transform);
        
        
        
        //Leftpaw front 2
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 15.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-9.3, 0, -37.5));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_front2.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-9.3, 0, -45));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_front2_sphere.setModelTransformation(transform);
        
        //Leftpaw front 3
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 15.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-7.3, 0, -37.5));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_front3.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-7.3, 0, -45));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_front3_sphere.setModelTransformation(transform);
        
        //Leftpaw front 4
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 15.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-5.3, 0, -37.5));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_front4.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-5.3, 0, -45));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_front4_sphere.setModelTransformation(transform);
        
        
        
        //Rightpaw front 2
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 15.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(9.3, 0, -37.5));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_front2.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(9.3, 0, -45));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_front2_sphere.setModelTransformation(transform);
        
        //Rightpaw front 3
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 15.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(7.3, 0, -37.5));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_front3.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(7.3, 0, -45));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_front3_sphere.setModelTransformation(transform);
        
        //Rightpaw front 4
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 15.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(5.3, 0, -37.5));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_front4.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(3, 3.0, 3.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(5.3, 0, -45));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_front4_sphere.setModelTransformation(transform);
        
        
        //Neck 
        scale = glm.scale(glm.mat4(1.0), glm.vec3(6, 2, 6.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 9, -20));
        transform = pos['*'](scale);
        this.neck.setModelTransformation(transform);
        
        
        //Head
        rot = glm.toMat4(glm.angleAxis(glm.radians(0.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(5.5, 6, 5.5));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 12, -21.5));
        transform = pos['*'](rot)['*'](scale);
        this.head.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(6, 6.0, 6.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(1, 12, -19));
        transform = pos['*'](scale);
        this.cone1.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(6, 6.0, 6.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-1, 12, -19));
        transform = pos['*'](scale);
        this.cone2.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(6, 6.0, 6.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(2.5, 12, -20));
        transform = pos['*'](scale);
        this.cone3.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(6, 6.0, 6.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-2.5, 12, -20));
        transform = pos['*'](scale);
        this.cone4.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(6, 6.0, 6.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(2.8, 12, -21.5));
        transform = pos['*'](scale);
        this.cone5.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(6, 6.0, 6.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-2.8, 12, -21.5));
        transform = pos['*'](scale);
        this.cone6.setModelTransformation(transform);        
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(6, 6.0, 6.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(2.5, 12, -22.5));
        transform = pos['*'](scale);
        this.cone7.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(6, 6.0, 6.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-2.5, 12, -22.5));
        transform = pos['*'](scale);
        this.cone8.setModelTransformation(transform); 
        
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(5.5, 5.5, 5.5));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 12.7, -22.5));
        transform = pos['*'](scale);
        this.face.setModelTransformation(transform); 
        
        
         //Ceiling borders
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2, 55, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(9.5, 5, 0));
        transform = pos['*'](rot)['*'](scale);
        this.ceiling_border_1.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2, 55, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-9.5, 5, 0));
        transform = pos['*'](rot)['*'](scale);
        this.ceiling_border_2.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,1.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2, 19, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 5, 27.5));
        transform = pos['*'](rot)['*'](scale);
        this.ceiling_border_3.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,1.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2, 19, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 5, -27.5));
        transform = pos['*'](rot)['*'](scale);
        this.ceiling_border_4.setModelTransformation(transform);
        
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2, 50, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(8.5, 6, 0));
        transform = pos['*'](rot)['*'](scale);
        this.ceiling_border_5.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2, 50, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-8.5, 6, 0));
        transform = pos['*'](rot)['*'](scale);
        this.ceiling_border_6.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,1.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2, 17, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 6, 25));
        transform = pos['*'](rot)['*'](scale);
        this.ceiling_border_7.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,1.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2, 17, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 6, -25));
        transform = pos['*'](rot)['*'](scale);
        this.ceiling_border_8.setModelTransformation(transform);
        
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2, 45, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(7.5, 7, 0));
        transform = pos['*'](rot)['*'](scale);
        this.ceiling_border_9.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2, 45, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-7.5, 7, 0));
        transform = pos['*'](rot)['*'](scale);
        this.ceiling_border_10.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,1.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2, 15, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 7, 22.5));
        transform = pos['*'](rot)['*'](scale);
        this.ceiling_border_11.setModelTransformation(transform);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,1.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2, 15, 2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 7, -22.5));
        transform = pos['*'](rot)['*'](scale);
        this.ceiling_border_12.setModelTransformation(transform);
        
        //Tail
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,1.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1.5, 8, 1.5));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(3, 0, 30));
        transform = pos['*'](rot)['*'](scale);
        this.tail.setModelTransformation(transform);
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0)){
        
        this.floor.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling2.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling3.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling4.draw(viewM, projectionM, this.shader, parentM);
        this.backwall.draw(viewM, projectionM, this.shader, parentM);

        this.leftwall.draw(viewM, projectionM, this.shader, parentM);
        this.leftwall1.draw(viewM, projectionM, this.shader, parentM);
        this.leftwall2.draw(viewM, projectionM, this.shader, parentM);
        this.leftwall3.draw(viewM, projectionM, this.shader, parentM);
        this.rightwall.draw(viewM, projectionM, this.shader, parentM);
        this.rightwall1.draw(viewM, projectionM, this.shader, parentM);
        this.rightwall2.draw(viewM, projectionM, this.shader, parentM);
        this.rightwall3.draw(viewM, projectionM, this.shader, parentM);

        this.frontwall_top.draw(viewM, projectionM, this.shader, parentM);
        this.frontwall_left.draw(viewM, projectionM, this.shader, parentM);
        this.frontwall_right.draw(viewM, projectionM, this.shader, parentM);

        this.leftpaw_back.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_back.draw(viewM, projectionM, this.shader, parentM);

        this.leftpaw_front1.draw(viewM, projectionM, this.shader, parentM);
        this.leftpaw_front2.draw(viewM, projectionM, this.shader, parentM);
        this.leftpaw_front3.draw(viewM, projectionM, this.shader, parentM);
        this.leftpaw_front4.draw(viewM, projectionM, this.shader, parentM);

        this.rightpaw_front1.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_front2.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_front3.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_front4.draw(viewM, projectionM, this.shader, parentM);
        
        
        this.leftpaw_back_sphere.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_back_sphere.draw(viewM, projectionM, this.shader, parentM);

        this.leftpaw_back_sphere2.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_back_sphere2.draw(viewM, projectionM, this.shader, parentM);
        
        this.leftpaw_front1_sphere2.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_front1_sphere2.draw(viewM, projectionM, this.shader, parentM);
        
        this.leftpaw_front1_sphere.draw(viewM, projectionM, this.shader, parentM);
        this.leftpaw_front2_sphere.draw(viewM, projectionM, this.shader, parentM);
        this.leftpaw_front3_sphere.draw(viewM, projectionM, this.shader, parentM);
        this.leftpaw_front4_sphere.draw(viewM, projectionM, this.shader, parentM);

        this.rightpaw_front1_sphere.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_front2_sphere.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_front3_sphere.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_front4_sphere.draw(viewM, projectionM, this.shader, parentM);
        
        this.head.draw(viewM, projectionM, this.shader, parentM);
        this.neck.draw(viewM, projectionM, this.shader, parentM);
        
        this.cone1.draw(viewM, projectionM, this.shader, parentM);
        this.cone2.draw(viewM, projectionM, this.shader, parentM);
        this.cone3.draw(viewM, projectionM, this.shader, parentM);
        this.cone4.draw(viewM, projectionM, this.shader, parentM);
        this.cone5.draw(viewM, projectionM, this.shader, parentM);
        this.cone6.draw(viewM, projectionM, this.shader, parentM);
        this.cone7.draw(viewM, projectionM, this.shader, parentM);
        this.cone8.draw(viewM, projectionM, this.shader, parentM);
        
        this.face.draw(viewM, projectionM, this.shader, parentM);
        
        this.ceiling_border_1.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling_border_2.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling_border_3.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling_border_4.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling_border_5.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling_border_6.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling_border_7.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling_border_8.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling_border_9.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling_border_10.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling_border_11.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling_border_12.draw(viewM, projectionM, this.shader, parentM);
        this.tail.draw(viewM, projectionM, this.shader, parentM);
    }
}


</script>

In [ ]:
%%html
<canvas id="myCanvas5" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp5">

class myapp5 extends DEECapp{    
    counter = 0;
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader-texture").text;
        var vertsrc = document.getElementById("my-vertex-shader-texture").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.sphynx = new sphynx(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.sphynx.setShader(this.shaderprog);
        
        this.projectionM = glm.perspective(glm.radians(45), 1.0, 0.1, 1000.0);
        this.viewM = glm.lookAt(glm.vec3(-30, 30, -85), glm.vec3(0, 0, -20), glm.vec3(0, 1, 0));
        
        
        this.sphynxTexture = new CGRAtexture(this.gl);
        this.sphynxTexture.load("Textures/sand2.jpg");
        
        this.floorTexture = new CGRAtexture(this.gl);
        this.floorTexture.load("Textures/chess2.jpg");
        
        this.sphynx.setTexture(this.sphynxTexture, this.floorTexture);
        
    }
    
    
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        const I = glm.mat4(1.0);

        this.sphynx.draw(this.viewM, this.projectionM, I);
        }

}


var app5 = new myapp5('myCanvas5');

app5.run();

</script>

In [ ]:
%%html
<script id="column">

class column extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.colum = new cylinderT(this.gl, color1, color2);
        this.top = new coneT(this.gl, color2, color1);
        this.bottom = new coneT(this.gl, color1, color2);

        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.colum.setShader(shaderprog);
        this.bottom.setShader(shaderprog);
        this.top.setShader(shaderprog);
    }
    
    setTexture(columnTex) {
        this.columnTex  = columnTex;

        this.colum.setTexture(columnTex);
        this.top.setTexture(columnTex);
        this.bottom.setTexture(columnTex);
    }
    
    create_obj(){
        
        var transform = glm.scale(glm.mat4(1.0), glm.vec3(1, 2, 1));
        this.colum.setModelTransformation(transform);
        
        var scale = glm.scale(glm.mat4(1.0), glm.vec3(1.5, 1, 1.5));
        var pos = glm.translate(glm.mat4(1.0), glm.vec3(0, -0.5, 0));
        transform = pos['*'](scale);
        this.bottom.setModelTransformation(transform); 
        
        var rot = glm.toMat4(glm.angleAxis(glm.radians(180.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1.5, 1, 1.5));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 0.5, 0));
        transform = pos['*'](rot)['*'](scale);
        this.top.setModelTransformation(transform); 
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0)){
        
        this.colum.draw(viewM, projectionM, this.shader, parentM);
        this.top.draw(viewM, projectionM, this.shader, parentM);
        this.bottom.draw(viewM, projectionM, this.shader, parentM);
    }
}


</script>

In [ ]:
%%html
<script id="obelisk">

class obelisk extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.colum = new cubeT(this.gl, color1, color2);
        this.top = new pyramidT(this.gl, color1, color2);
        this.bottom = new pyramidT(this.gl, color1, color2);

        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.colum.setShader(shaderprog);
        this.bottom.setShader(shaderprog);
        this.top.setShader(shaderprog);
    }
    
    setTexture(obeliskTex) {
        this.obeliskTex  = obeliskTex;

        this.colum.setTexture(obeliskTex);
        this.top.setTexture(obeliskTex);
        this.bottom.setTexture(obeliskTex);
    }
    
    create_obj(){
        
        var scale = glm.scale(glm.mat4(1.0), glm.vec3(1, 7, 1));
        var pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 3.5, 0));
        var transform = pos['*'](scale);
        this.colum.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1.3, 8, 1.3));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 4, 0));
        transform = pos['*'](scale);
        this.bottom.setModelTransformation(transform); 
        
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 7.5, 0));
        transform = pos;
        this.top.setModelTransformation(transform); 
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0)){
        
        this.colum.draw(viewM, projectionM, this.shader, parentM);
        this.top.draw(viewM, projectionM, this.shader, parentM);
        this.bottom.draw(viewM, projectionM, this.shader, parentM);
    }
}


</script>

In [ ]:
%%html
<script id="bench">

class bench extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.foot = new cubeT(this.gl, color1, color2);
        this.base = new cubeT(this.gl, color1, color2);
        this.top = new cubeT(this.gl, color1, color2);

        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.foot.setShader(shaderprog);
        this.base.setShader(shaderprog);
        this.top.setShader(shaderprog);
    }
    
    setTexture(benchTex, benchFeetTex) {
        this.benchTex  = benchTex;
        this.benchFeetTex  = benchFeetTex;

        this.foot.setTexture(benchFeetTex);
        this.base.setTexture(benchTex);
        this.top.setTexture(benchTex);
    }
    
    create_obj(){
        
        var scale = glm.scale(glm.mat4(1.0), glm.vec3(2, 0.4, 0.5));
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0,1.0,0.0)));
        var pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 0.2, 0));
        var transform = pos['*'](rot['*'](scale));
        this.foot.setModelTransformation(transform);
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2.5, 1.0, 0.2));
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0,1.0,0.0)));
        let rot2 = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0,0.0,1.0)));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0, 0.5, 0));
        transform = pos['*'](rot2['*'](rot['*'](scale)));
        this.base.setModelTransformation(transform); 
        
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2.5, 0.5, 0.2));
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0,1.0,0.0)));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.4, 0.85, 0));
        transform = pos['*'](rot['*'](scale));
        this.top.setModelTransformation(transform); 
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0)){
        
        this.foot.draw(viewM, projectionM, this.shader, parentM);
        this.base.draw(viewM, projectionM, this.shader, parentM);
        this.top.draw(viewM, projectionM, this.shader, parentM);
    }
}


</script>

In [ ]:
%%html
<script id="palmTree">

class palmTree extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.color1 = color1;
        this.color2 = color2;
        
        this.body1 = new cylinderT(this.gl, color1, color2);
        this.body2 = new sphereT(this.gl, color1, color2);
        
        this.leafs = [];
        
        this.create_obj();
    }
    
    setShader(shaderprog){       
        this.shader = shaderprog;
        
        this.body1.setShader(shaderprog);
        this.body2.setShader(shaderprog);
        
        for (let l of this.leafs) l.setShader(shaderprog);
    }
    
    setTexture(woodTex, leafTex) {
        this.woodTex  = woodTex;
        this.leafTex  = leafTex;

        this.body1.setTexture(woodTex);
        this.body2.setTexture(woodTex);
        
        for (let l of this.leafs) {
            l.setTexture(leafTex);
        }
    }
    
    create_obj(){
        
        const c1 = this.color1;
        const c2 = this.color2;
        
        // body1
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,0.0)));
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 7.0, 1.0));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 3.5, 0.0));
        let transform = pos['*'](rot)['*'](scale);
        this.body1.setModelTransformation(transform);
        
        // body2
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1.5, 2.0, 1.5));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 7.0, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.body2.setModelTransformation(transform);
        
        // leafs
        const leafScales = [
            glm.vec3(5.0, 1.0, 1.0),
            glm.vec3(5.0, 1.0, 1.0),
            glm.vec3(5.0, 1.0, 1.0),
            glm.vec3(5.0, 1.0, 1.0),
            glm.vec3(5.0, 1.0, 1.0),
            glm.vec3(5.0, 1.0, 1.0),
            glm.vec3(5.0, 1.0, 1.0),
            glm.vec3(5.0, 1.0, 1.0)
        ];
        
        const leafAnglesX = [15, 15, 15, 15, 15, 15, 15, 15]; // degrees around X-axis
        const leafAnglesY = [0, 45, 90, 135, 180, 225, 270, 315]; // degrees around Y-axis

        this.leafs = [];

        for (let i = 0; i < leafScales.length; i++) {
            const c1 = [0.278,0.404,0.11];
            const c2 = [0.49,0.635,0.325];
            
            const leaf = new diskT(this.gl, c1, c2);

            const rotX = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0, 1.0, 0.0)));
            const rotX2 = glm.toMat4(glm.angleAxis(glm.radians(leafAnglesX[i]), glm.vec3(1.0, 0.0, 0.0)));
            const rotY = glm.toMat4(glm.angleAxis(glm.radians(leafAnglesY[i]), glm.vec3(0.0, 1.0, 0.0)));

            const scale = glm.scale(glm.mat4(1.0), leafScales[i]);
            const pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 8.0, 0.0));
            const out = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, 1.75));

            const baseTransform = pos['*'](rotY)['*'](rotX2)['*'](out)['*'](rotX)['*'](scale);

            leaf.setModelTransformation(baseTransform);
            this.leafs.push(leaf);
        }
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
          const base = parentM['*'](this.modelMat);
        
          this.body1.draw(viewM, projectionM, this.shader, base);
          this.body2.draw(viewM, projectionM, this.shader, base);
        
          for (let l of this.leafs) l.draw(viewM, projectionM, this.shader, base);
     }

}


</script>

In [ ]:
%%html
<script id="turnstileArms">

class turnstileArms extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.color1 = color1;
        this.color2 = color2;
        
        this.origin = new sphereT(this.gl, [0.0, 0.0, 0.0], [0.490, 0.477, 0.451]);
        
        this.arms = [];
        
        this.create_obj();
    }
    
    setShader(shaderprog){       
        this.shader = shaderprog;
        
        this.origin.setShader(shaderprog);
        
        for (let a of this.arms) a.setShader(shaderprog);
    }
    
    setTexture(metalTex) {
        this.metalTex  = metalTex;

        this.origin.setTexture(metalTex);
        
        for (let a of this.arms) {
            a.setTexture(metalTex);
        }
    }
    
    create_obj(){
        
        const c1 = this.color1;
        const c2 = this.color2;
        
        // origin
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,0.0)));
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 1.0, 1.0));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 2.25, 0.4));
        let transform = pos['*'](rot)['*'](scale);
        this.origin.setModelTransformation(transform);
        
        // arms
        const armPositions = [
            glm.vec3(0.0, 2.25, 0.0),   // center pivot (same for all arms)
            glm.vec3(0.0, 2.5, 0.25),
            glm.vec3(0.0, 2.5, 0.25)
        ];

        const armAnglesX = [0, 60, 60]; // degrees around X-axis
        const armAnglesY = [0, 60, -60]; // degrees around Y-axis

        this.arms = [];

        for (let i = 0; i < armPositions.length; i++) {
            const arm = new cylinderT(this.gl, c1, c2);

            const rotX = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(1.0, 0.0, 0.0)));
            const rotX2 = glm.toMat4(glm.angleAxis(glm.radians(armAnglesX[i]), glm.vec3(1.0, 0.0, 0.0)));
            const rotY = glm.toMat4(glm.angleAxis(glm.radians(armAnglesY[i]), glm.vec3(0.0, 1.0, 0.0)));

            const scale = glm.scale(glm.mat4(1.0), glm.vec3(0.3, 2.75, 0.3));
            const pos = glm.translate(glm.mat4(1.0), armPositions[i]);
            const out = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, 1.75));

            const baseTransform = pos['*'](rotY)['*'](rotX2)['*'](out)['*'](rotX)['*'](scale);

            arm.setModelTransformation(baseTransform);
            this.arms.push(arm);
        }
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
          const base = parentM['*'](this.modelMat);
        
          this.origin.draw(viewM, projectionM, this.shader, base);
        
          for (let a of this.arms) a.draw(viewM, projectionM, this.shader, base);
     }

}


</script>

In [ ]:
%%html
<script id="turnstile">

class turnstile extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.base1 = new cylinderT(this.gl, color1, color2);
        this.base2 = new cylinderT(this.gl, color1, color2);
        this.base3 = new cubeT(this.gl, color1, color2);

        this.turnstileArms = new turnstileArms(this.gl, color1, color2);
        
        this.create_obj();
    }
    
    setShader(shaderprog){       
        this.shader = shaderprog;
        
        this.base1.setShader(shaderprog);
        this.base2.setShader(shaderprog);
        this.base3.setShader(shaderprog);
        
        this.turnstileArms.setShader(shaderprog);
    }
    
    setTexture(metalTex) {
        this.metalTex  = metalTex;

        this.base1.setTexture(metalTex);
        this.base2.setTexture(metalTex);
        this.base3.setTexture(metalTex);
        
        this.turnstileArms.setTexture(metalTex);
    }
    
    create_obj(){
        
        // base1
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,0.0)));
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 4.0, 1.0));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(1.5, 1.0, 0.0));
        let transform = pos['*'](rot)['*'](scale);
        this.base1.setModelTransformation(transform);
        
        // base2
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 4.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-1.5, 1.0, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.base2.setModelTransformation(transform);
        
        // base3
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,1.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.75, 2.75, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 2.65, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.base3.setModelTransformation(transform);
        
        // turnstileArms
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0, 0.0, 0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 1.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.turnstileArms.setModelTransformation(transform);
              
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
          const base = parentM['*'](this.modelMat);
        
          this.base1.draw(viewM, projectionM, this.shader, base);
          this.base2.draw(viewM, projectionM, this.shader, base);
          this.base3.draw(viewM, projectionM, this.shader, base);
        
          const center = glm.vec3(0.0, 2.25, 0.4); // center of rotation
          const I   = glm.mat4(1.0);
          const T   = glm.translate(I, center);
          const Tin = glm.translate(I, glm.vec3(-center.x, -center.y, -center.z));
        
          const tilt = 45.0;
          const R   = glm.toMat4(glm.angleAxis(glm.radians(tilt), glm.vec3(1.0, 0.0, 0.0)));
          const Rin   = glm.toMat4(glm.angleAxis(glm.radians(-tilt), glm.vec3(1.0, 0.0, 0.0)));

          const ang  = (counter/3);  //~18 second loop  (1 times slower than a normal loop, 6.28s)
          const Rz   = glm.toMat4(glm.angleAxis(ang, glm.vec3(0.0, 0.0, 1.0)));

          const armsParent = base['*'](T)['*'](R)['*'](Rz)['*'](Rin)['*'](Tin);

          this.turnstileArms.draw(viewM, projectionM, armsParent, counter);
     }

}


</script>

### Main

O programa possui três modos, dois de passageiro e um de visitante. Clicando v, a vista passa a ser de um passageiro a olhar para fora. Clicando b, a vista passa a ser de um passageiro a olhar para dentro. E o modo normal é o de visitante, controlado por 'wasd'. 


In [ ]:
%%html
<canvas id="myCanvas2" width="800" height="800" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myapp2 extends DEECapp{
    
    orientation = 90; // orientation of the camera
    mode = 0; // Visitor == 0 or Train == 1
    shape = 0; //Shape for the train wheels
    
    //Double because the objects are from -0.5 to 0.5
    track_length = 70*2;
    track_width = 35*2;
    
    //~30 second loop  (5 times slower than a normal loop, 6.28s)
    loopSpeed = 5.0;  
    loopDuration = this.loopSpeed*2*Math.PI;  
    
    pauseDuration = 4.0;
    
    cameraPos = glm.vec3(0, 1, -150); //Initial position of the camera looking to center 
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader-texture").text;
        var vertsrc = document.getElementById("my-vertex-shader-texture").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        //Background color
        this.gl.clearColor(0, 0, 1, 0.2);        
        
        this.projectionM = glm.perspective(glm.radians(45),1,0.1,1000);
        this.viewM = glm.lookAt(this.cameraPos,glm.vec3(this.cameraPos.x+Math.cos(glm.radians(this.orientation)),1,this.cameraPos.z+Math.sin(glm.radians(this.orientation))),glm.vec3(0,1,0));
        
        //===========================================================
        // Initialize objects
        
        //Paintings
        
        
        
        //Lamp
        
        //Sphynx train station
        this.sphynx = new sphynx(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.sphynx.setShader(this.shaderprog);
        
        this.sphynxTexture = new CGRAtexture(this.gl);
        this.sphynxTexture.load("Textures/sand2.jpg");
        this.floorTexture = new CGRAtexture(this.gl);
        this.floorTexture.load("Textures/chess2.jpg");
        this.sphynx.setTexture(this.sphynxTexture, this.floorTexture);
        
        //Pyramids
        this.bigpyramid1 = new pyramidT(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.bigpyramid1.setShader(this.shaderprog);
        
        this.bigpyramid2 = new pyramidT(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.bigpyramid2.setShader(this.shaderprog);
        
        this.bigpyramid3 = new pyramidT(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.bigpyramid3.setShader(this.shaderprog);
        
        this.pyramidTexture = new CGRAtexture(this.gl);
        this.pyramidTexture.load("Textures/sandstone.jpg");
        this.bigpyramid1.setTexture(this.pyramidTexture);
        this.bigpyramid2.setTexture(this.pyramidTexture);
        this.bigpyramid3.setTexture(this.pyramidTexture);
        
        
        //Sun
        this.sun = new sphereT(this.gl, [0.970,0.558,0.0194], [0.970,0.558,0.0194]);
        this.sun.setShader(this.shaderprog);
        
        this.sunTexture = new CGRAtexture(this.gl);
        this.sunTexture.load("Textures/sun3.jpg");
        this.moonTexture = new CGRAtexture(this.gl);
        this.moonTexture.load("Textures/moon.jpg");
        this.sun.setTexture(this.sunTexture);
        
        //Doors
        this.leftdoor = new cubeT(this.gl,[0.450, 0.301, 0.00450],[0.450, 0.301, 0.00450]);
        this.leftdoor.setShader(this.shaderprog);
        
        this.rightdoor = new cubeT(this.gl,[0.450, 0.301, 0.00450],[0.450, 0.301, 0.00450]);
        this.rightdoor.setShader(this.shaderprog);
        
        this.doorTexture = new CGRAtexture(this.gl);
        this.doorTexture.load("Textures/wood_door.jpg");
        this.leftdoor.setTexture(this.doorTexture);
        this.rightdoor.setTexture(this.doorTexture);
        
        
        //Lakes
        this.lake = new diskT(this.gl, [0.460, 0.866, 0.920],[1,1,1]);
        this.lake.setShader(this.shaderprog);
        
        this.oasis = new diskT(this.gl, [0.356, 0.690, 0.235], [0.356, 0.690, 0.235]);
        this.oasis.setShader(this.shaderprog);
        
        this.lake2 = new diskT(this.gl, [0.460, 0.866, 0.920],[1,1,1]);
        this.lake2.setShader(this.shaderprog);
        
        this.oasis2 = new diskT(this.gl, [0.356, 0.690, 0.235], [0.356, 0.690, 0.235]);
        this.oasis2.setShader(this.shaderprog);
        
        this.lake3 = new diskT(this.gl, [0.460, 0.866, 0.920],[1,1,1]);
        this.lake3.setShader(this.shaderprog);
        
        this.oasis3 = new diskT(this.gl, [0.356, 0.690, 0.235], [0.356, 0.690, 0.235]);
        this.oasis3.setShader(this.shaderprog);
        
        this.waterTexture = new CGRAtexture(this.gl);
        this.waterTexture.load("Textures/water.jpg");
        this.grassTexture = new CGRAtexture(this.gl);
        this.grassTexture.load("Textures/grass.jpg");
        this.lake.setTexture(this.waterTexture);
        this.oasis.setTexture(this.grassTexture);
        this.lake2.setTexture(this.waterTexture);
        this.oasis2.setTexture(this.grassTexture);
        this.lake3.setTexture(this.waterTexture);
        this.oasis3.setTexture(this.grassTexture);
        
        //Floor and track
        this.floor = new diskT(this.gl, [0.886, 0.792, 0.463], [0.886, 0.792, 0.463]);
        this.floor.setShader(this.shaderprog);
        
        this.inner_floor = new diskT(this.gl, [0.886, 0.792, 0.463], [0.886, 0.792, 0.463], 60);
        this.inner_floor.setShader(this.shaderprog);
        
        this.track = new diskT(this.gl, [0.6,0.6,0.6], [0.6,0.6,0.6], 60);
        this.track.setShader(this.shaderprog);
        
        this.groundTexture = new CGRAtexture(this.gl);
        this.groundTexture.load("Textures/sand8.jpg");
        this.floor.setTexture(this.groundTexture);
        this.inner_floor.setTexture(this.groundTexture);
        
        this.trackTexture = new CGRAtexture(this.gl);
        this.trackTexture.load("Textures/gravel2.jpg");
        this.track.setTexture(this.trackTexture);
        
        //Train
        this.train = new train(this.gl, [0, 0, 0], [0.450, 0.00450, 0.0193]);
        this.train.setShader(this.shaderprog);
        
        this.engineTexture = new CGRAtexture(this.gl);
        this.engineTexture.load("Textures/locomotive_metal2.jpg");
        
        this.bodyTexture = new CGRAtexture(this.gl);
        this.bodyTexture.load("Textures/train_metal.jpg");

        this.cgraTexture = new CGRAtexture(this.gl);
        this.cgraTexture.load("Textures/cgra_texture2.jpg");
        
        this.wheelTexture = new CGRAtexture(this.gl);
        this.wheelTexture.load("Textures/locomotive_metal2.jpg");
        
        this.train.setTexture(this.engineTexture, this.bodyTexture, this.wheelTexture, this.cgraTexture);
        
        //Columns 
        this.column1 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.column1.setShader(this.shaderprog);
        
        this.column2 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.column2.setShader(this.shaderprog);
        
        this.column3 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.column3.setShader(this.shaderprog);
        
        this.column4 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.column4.setShader(this.shaderprog);
        
        this.column5 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.column5.setShader(this.shaderprog);
        
        this.column6 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.column6.setShader(this.shaderprog);
        
        this.column7 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.column7.setShader(this.shaderprog);
        
        this.column8 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.column8.setShader(this.shaderprog);
        
        this.columnTexture = new CGRAtexture(this.gl);
        this.columnTexture.load("Textures/sandstone.jpg");
        this.column1.setTexture(this.columnTexture);
        this.column2.setTexture(this.columnTexture);
        this.column3.setTexture(this.columnTexture);
        this.column4.setTexture(this.columnTexture);
        this.column5.setTexture(this.columnTexture);
        this.column6.setTexture(this.columnTexture);
        this.column7.setTexture(this.columnTexture);
        this.column8.setTexture(this.columnTexture);
        
        //Tunnel columns
        this.tunnel_column1 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.tunnel_column1.setShader(this.shaderprog);
        
        this.tunnel_column2 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.tunnel_column2.setShader(this.shaderprog);
        
        this.tunnel_column3 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.tunnel_column3.setShader(this.shaderprog);
        
        this.tunnel_column4 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.tunnel_column4.setShader(this.shaderprog);
        
        this.tunnel_column1.setTexture(this.columnTexture);
        this.tunnel_column2.setTexture(this.columnTexture);
        this.tunnel_column3.setTexture(this.columnTexture);
        this.tunnel_column4.setTexture(this.columnTexture);
        
        //Bench
        this.bench1 = new bench(this.gl,[0.450, 0.301, 0.00450],[0.42,0.286,0.169]);
        this.bench1.setShader(this.shaderprog);
        
        this.bench2 = new bench(this.gl,[0.450, 0.301, 0.00450],[0.42,0.286,0.169]);
        this.bench2.setShader(this.shaderprog);
        
        this.bench3 = new bench(this.gl,[0.450, 0.301, 0.00450],[0.42,0.286,0.169]);
        this.bench3.setShader(this.shaderprog);
        
        this.bench4 = new bench(this.gl,[0.450, 0.301, 0.00450],[0.42,0.286,0.169]);
        this.bench4.setShader(this.shaderprog);
        
        this.bench5 = new bench(this.gl,[0.450, 0.301, 0.00450],[0.42,0.286,0.169]);
        this.bench5.setShader(this.shaderprog);
        
        this.bench6 = new bench(this.gl,[0.450, 0.301, 0.00450],[0.42,0.286,0.169]);
        this.bench6.setShader(this.shaderprog);
        
        this.bench7 = new bench(this.gl,[0.450, 0.301, 0.00450],[0.42,0.286,0.169]);
        this.bench7.setShader(this.shaderprog);
        
        this.bench8 = new bench(this.gl,[0.450, 0.301, 0.00450],[0.42,0.286,0.169]);
        this.bench8.setShader(this.shaderprog);
        
        this.benchWoodTexture = new CGRAtexture(this.gl);
        this.benchWoodTexture.load("Textures/wood_bench.jpg");
        this.footMetalTexture = new CGRAtexture(this.gl);
        this.footMetalTexture.load("Textures/iron.jpg");
        this.bench1.setTexture(this.benchWoodTexture, this.footMetalTexture);
        this.bench2.setTexture(this.benchWoodTexture, this.footMetalTexture);
        this.bench3.setTexture(this.benchWoodTexture, this.footMetalTexture);
        this.bench4.setTexture(this.benchWoodTexture, this.footMetalTexture);
        this.bench5.setTexture(this.benchWoodTexture, this.footMetalTexture);
        this.bench6.setTexture(this.benchWoodTexture, this.footMetalTexture);
        this.bench7.setTexture(this.benchWoodTexture, this.footMetalTexture);
        this.bench8.setTexture(this.benchWoodTexture, this.footMetalTexture);
        
        
        
        //Camels
        this.camel = new camel(this.gl, [0.439, 0.314, 0.184], [0.757, 0.604, 0.420]);
        this.camel.setShader(this.shaderprog);
        
        this.camel2 = new camel(this.gl, [0.439, 0.314, 0.184], [0.757, 0.604, 0.420]);
        this.camel2.setShader(this.shaderprog);
        
        this.camel3 = new camel(this.gl, [0.439, 0.314, 0.184], [0.757, 0.604, 0.420]);
        this.camel3.setShader(this.shaderprog);
        
        this.camel4 = new camel(this.gl, [0.439, 0.314, 0.184], [0.757, 0.604, 0.420]);
        this.camel4.setShader(this.shaderprog);
        
        this.camel5 = new camel(this.gl, [0.439, 0.314, 0.184], [0.757, 0.604, 0.420]);
        this.camel5.setShader(this.shaderprog);
        
        this.camel6 = new camel(this.gl, [0.439, 0.314, 0.184], [0.757, 0.604, 0.420]);
        this.camel6.setShader(this.shaderprog);
        
        this.camel7 = new camel(this.gl, [0.439, 0.314, 0.184], [0.757, 0.604, 0.420]);
        this.camel7.setShader(this.shaderprog);
        
        this.camel8 = new camel(this.gl, [0.439, 0.314, 0.184], [0.757, 0.604, 0.420]);
        this.camel8.setShader(this.shaderprog);
        
        
        this.moon_camel = new camel(this.gl, [0.439, 0.314, 0.184], [0.757, 0.604, 0.420]);
        this.moon_camel.setShader(this.shaderprog);
        
        this.camelTexture = new CGRAtexture(this.gl);
        this.camelTexture.load("Textures/camel1.jpg");
        this.camelTexture2 = new CGRAtexture(this.gl);
        this.camelTexture2.load("Textures/camel5.jpg");
        this.camel.setTexture(this.camelTexture);
        this.camel2.setTexture(this.camelTexture);
        this.camel3.setTexture(this.camelTexture);
        this.camel4.setTexture(this.camelTexture2);
        this.camel5.setTexture(this.camelTexture2);
        this.camel6.setTexture(this.camelTexture);
        this.camel7.setTexture(this.camelTexture2);
        this.camel8.setTexture(this.camelTexture2);
        this.moon_camel.setTexture(this.camelTexture2);
        
        //Lamps
        this.lamp1 = new lampPost(this.gl, [0.190, 0.178, 0.154], [0.480, 0.461, 0.422]);
        this.lamp1.setShader(this.shaderprog);
        
        this.lamp2 = new lampPost(this.gl, [0.190, 0.178, 0.154], [0.480, 0.461, 0.422]);
        this.lamp2.setShader(this.shaderprog);
        
        this.lampPostTexture = new CGRAtexture(this.gl);
        this.lampPostTexture.load("Textures/iron.jpg");
        this.lampTexture = new CGRAtexture(this.gl);
        this.lampTexture.load("Textures/glass.jpg");
        this.lamp1.setTexture(this.lampPostTexture, this.lampTexture);
        this.lamp2.setTexture(this.lampPostTexture, this.lampTexture);
        
        //Turnstile
        
        this.turnstile1 = new turnstile(this.gl, [0.190, 0.178, 0.154], [0.480, 0.461, 0.422]);
        this.turnstile1.setShader(this.shaderprog);
        
        this.turnstile2 = new turnstile(this.gl, [0.190, 0.178, 0.154], [0.480, 0.461, 0.422]);
        this.turnstile2.setShader(this.shaderprog);
        
        this.turnstile3 = new turnstile(this.gl, [0.190, 0.178, 0.154], [0.480, 0.461, 0.422]);
        this.turnstile3.setShader(this.shaderprog);
        
        this.turnstile4 = new turnstile(this.gl, [0.190, 0.178, 0.154], [0.480, 0.461, 0.422]);
        this.turnstile4.setShader(this.shaderprog);
        
        this.turnstileTexture = new CGRAtexture(this.gl);
        this.turnstileTexture.load("Textures/metal1.jpg");
        this.turnstile1.setTexture(this.turnstileTexture);
        this.turnstile2.setTexture(this.turnstileTexture);
        this.turnstile3.setTexture(this.turnstileTexture);
        this.turnstile4.setTexture(this.turnstileTexture);
        
        //Palmtree
        this.palmtrees = [];
        this.palmtrees_trans = [];
        
        
        this.trunkTexture = new CGRAtexture(this.gl);
        this.trunkTexture.load("Textures/tree_trunk.jpg");
        this.leafTexture = new CGRAtexture(this.gl);
        this.leafTexture.load("Textures/tree_leaf2.jpg");
        for (let i = 0; i < 30; i++) {
            this.palmtrees[i] = new palmTree(this.gl, [0.42,0.286,0.169], [0.714,0.592,0.49]);
            this.palmtrees[i].setShader(this.shaderprog);
            
            this.palmtrees[i].setTexture(this.trunkTexture, this.leafTexture);
        }
        
        //Extra columns
        this.extra_column1 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.extra_column1.setShader(this.shaderprog);
        this.extra_column2 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.extra_column2.setShader(this.shaderprog);
        this.extra_column3 = new column(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.extra_column3.setShader(this.shaderprog);
        
        this.extraTexture = new CGRAtexture(this.gl);
        this.extraTexture.load("Textures/sandstone2.jpg");
        this.extra_column1.setTexture(this.extraTexture);
        this.extra_column2.setTexture(this.extraTexture);
        this.extra_column3.setTexture(this.extraTexture);
        
        this.obelisk = new obelisk(this.gl, [0.800,0.570,0.0320], [0.950,0.816,0.333]);
        this.obelisk.setShader(this.shaderprog);
        
        this.obelisk.setTexture(this.columnTexture);
        
        //===========================================================
        
        //===========================================================
        //Translations
        this.trans_bigpyramid1 = glm.translate(glm.mat4(1.0),glm.vec3(700,27,650));
        this.trans_bigpyramid2 = glm.translate(glm.mat4(1.0),glm.vec3(500,37,550));
        this.trans_bigpyramid3 = glm.translate(glm.mat4(1.0),glm.vec3(300,17,450));
        
        this.trans_sun = glm.translate(glm.mat4(1.0),glm.vec3(400,200,700));
        
        this.trans_leftdoor = glm.translate(glm.mat4(1.0),glm.vec3(-1.,1,-79.9));
        this.trans_rightdoor = glm.translate(glm.mat4(1.0),glm.vec3(2,1,-80.9));
        
        
        //Floor and track
        this.trans_track = glm.translate(glm.mat4(1.0),glm.vec3(0,0.01,0));
        this.trans_inner_floor = glm.translate(glm.mat4(1.0),glm.vec3(0,0.02,0));
        
        //Sphynx
        this.trans_sphynx = glm.translate(glm.mat4(1.0),glm.vec3(0,0,-50));
        
        //columns
        this.trans1_column = glm.translate(glm.mat4(1.0),glm.vec3(5,2,-40));
        this.trans2_column = glm.translate(glm.mat4(1.0),glm.vec3(-5,2,-40));
        this.trans3_column = glm.translate(glm.mat4(1.0),glm.vec3(4,2,-53));
        this.trans4_column = glm.translate(glm.mat4(1.0),glm.vec3(4,2,-63));
        this.trans5_column = glm.translate(glm.mat4(1.0),glm.vec3(4,2,-73));
        this.trans6_column = glm.translate(glm.mat4(1.0),glm.vec3(-4,2,-53));
        this.trans7_column = glm.translate(glm.mat4(1.0),glm.vec3(-4,2,-63));
        this.trans8_column = glm.translate(glm.mat4(1.0),glm.vec3(-4,2,-73));
        
        //Tunnel columns
        this.trans1_tunnel_column = glm.translate(glm.mat4(1.0),glm.vec3(10,2,-28));
        this.trans2_tunnel_column = glm.translate(glm.mat4(1.0),glm.vec3(-10,2,-28));
        this.trans3_tunnel_column = glm.translate(glm.mat4(1.0),glm.vec3(10,2,-36));
        this.trans4_tunnel_column = glm.translate(glm.mat4(1.0),glm.vec3(-10,2,-36));
        
        //bench
        this.trans1_bench = glm.translate(glm.mat4(1.0),glm.vec3(4,0,-56));
        this.trans2_bench = glm.translate(glm.mat4(1.0),glm.vec3(-4,0,-56));
        this.trans3_bench = glm.translate(glm.mat4(1.0),glm.vec3(4,0,-60));
        this.trans4_bench = glm.translate(glm.mat4(1.0),glm.vec3(-4,0,-60));
        this.trans5_bench = glm.translate(glm.mat4(1.0),glm.vec3(4,0,-66));
        this.trans6_bench = glm.translate(glm.mat4(1.0),glm.vec3(-4,0,-66));
        this.trans7_bench = glm.translate(glm.mat4(1.0),glm.vec3(4,0,-70));
        this.trans8_bench = glm.translate(glm.mat4(1.0),glm.vec3(-4,0,-70));
        
        //Camel
        this.trans_camel = glm.translate(glm.mat4(1.0),glm.vec3(0,1.2,0));
        this.trans_camel2 = glm.translate(glm.mat4(1.0),glm.vec3(3,0.6,2));
        this.trans_camel3 = glm.translate(glm.mat4(1.0),glm.vec3(-10,1.2,8));
        
        this.trans_camel4 = glm.translate(glm.mat4(1.0),glm.vec3(55,0.6,40));
        this.trans_camel5 = glm.translate(glm.mat4(1.0),glm.vec3(57,1.2,42));
        this.trans_camel6 = glm.translate(glm.mat4(1.0),glm.vec3(-40,0.6,55));
        
        this.trans_camel7 = glm.translate(glm.mat4(1.0),glm.vec3(-65,1.2,-54));
        this.trans_camel8 = glm.translate(glm.mat4(1.0),glm.vec3(-50,1.2,50));
        
        this.trans_moon_camel = glm.translate(glm.mat4(1.0),glm.vec3(400,-226,700));
        
        this.trans_lake = glm.translate(glm.mat4(1.0),glm.vec3(0,0.05,6));
        this.trans_oasis = glm.translate(glm.mat4(1.0),glm.vec3(0,0.04,6));
        
        this.trans_lake2 = glm.translate(glm.mat4(1.0),glm.vec3(-72,0.05,-55));
        this.trans_oasis2 = glm.translate(glm.mat4(1.0),glm.vec3(-72,0.04,-55));
        
        this.trans_lake3 = glm.translate(glm.mat4(1.0),glm.vec3(-48,0.05,46));
        this.trans_oasis3 = glm.translate(glm.mat4(1.0),glm.vec3(-48,0.04,46));
        
        
        this.trans_lamp1 = glm.translate(glm.mat4(1.0),glm.vec3(3,0,-81));
        this.trans_lamp2 = glm.translate(glm.mat4(1.0),glm.vec3(-3,0,-81));
        
        this.trans1_turnstile = glm.translate(glm.mat4(1.0),glm.vec3(-2,0,-73));
        this.trans2_turnstile = glm.translate(glm.mat4(1.0),glm.vec3(2,0,-73));
        this.trans3_turnstile = glm.translate(glm.mat4(1.0),glm.vec3(-6,0,-73));
        this.trans4_turnstile = glm.translate(glm.mat4(1.0),glm.vec3(6,0,-73));
        
        this.palmtrees_trans[0] = glm.translate(glm.mat4(1.0),glm.vec3(50,0,50));
        this.palmtrees_trans[1] = glm.translate(glm.mat4(1.0),glm.vec3(50,0,-50));
        this.palmtrees_trans[2] = glm.translate(glm.mat4(1.0),glm.vec3(-50,0,-50));
        this.palmtrees_trans[3] = glm.translate(glm.mat4(1.0),glm.vec3(-45,0,50));
        
        this.palmtrees_trans[4] = glm.translate(glm.mat4(1.0),glm.vec3(0,0,15));
        this.palmtrees_trans[5] = glm.translate(glm.mat4(1.0),glm.vec3(-5,0,0));
    
        this.palmtrees_trans[6] = glm.translate(glm.mat4(1.0),glm.vec3(3,0,-95));
        this.palmtrees_trans[7] = glm.translate(glm.mat4(1.0),glm.vec3(-6,0,-110));
        this.palmtrees_trans[8] = glm.translate(glm.mat4(1.0),glm.vec3(7,0,-120));
                                                                       
        this.palmtrees_trans[9] = glm.translate(glm.mat4(1.0),glm.vec3(-78,0,-57));
        this.palmtrees_trans[10] = glm.translate(glm.mat4(1.0),glm.vec3(-75,0,-48));
        
        this.palmtrees_trans[11] = glm.translate(glm.mat4(1.0),glm.vec3(40,0,-60));
        
        this.palmtrees_trans[12] = glm.translate(glm.mat4(1.0),glm.vec3(20,0,-75));
        this.palmtrees_trans[13] = glm.translate(glm.mat4(1.0),glm.vec3(-18,0,-85));
        
        this.palmtrees_trans[14] = glm.translate(glm.mat4(1.0),glm.vec3(4,0,50));
        this.palmtrees_trans[15] = glm.translate(glm.mat4(1.0),glm.vec3(20,0,45));
        this.palmtrees_trans[16] = glm.translate(glm.mat4(1.0),glm.vec3(-15,0,40));
        this.palmtrees_trans[17] = glm.translate(glm.mat4(1.0),glm.vec3(-7,0,47));
        
        
        this.palmtrees_trans[18] = glm.translate(glm.mat4(1.0),glm.vec3(83,0,-15));
        this.palmtrees_trans[19] = glm.translate(glm.mat4(1.0),glm.vec3(-83,0,-13));
        this.palmtrees_trans[20] = glm.translate(glm.mat4(1.0),glm.vec3(-88,0,30));
        this.palmtrees_trans[21] = glm.translate(glm.mat4(1.0),glm.vec3(90,0,3));
        this.palmtrees_trans[22] = glm.translate(glm.mat4(1.0),glm.vec3(87,0,20));
        this.palmtrees_trans[23] = glm.translate(glm.mat4(1.0),glm.vec3(-90,0,12));
        
        this.palmtrees_trans[24] = glm.translate(glm.mat4(1.0),glm.vec3(-18,0,-7));
        this.palmtrees_trans[25] = glm.translate(glm.mat4(1.0),glm.vec3(25,0,5));
        this.palmtrees_trans[26] = glm.translate(glm.mat4(1.0),glm.vec3(15,0,-12));
        this.palmtrees_trans[27] = glm.translate(glm.mat4(1.0),glm.vec3(37,0,-10));
        this.palmtrees_trans[28] = glm.translate(glm.mat4(1.0),glm.vec3(-33,0,-13));
        this.palmtrees_trans[29] = glm.translate(glm.mat4(1.0),glm.vec3(-24,0,15));
        
        
        this.trans1_extra_column = glm.translate(glm.mat4(1.0),glm.vec3(47,0,-52));
        this.trans2_extra_column = glm.translate(glm.mat4(1.0),glm.vec3(-50,2.2,41));
        this.trans3_extra_column = glm.translate(glm.mat4(1.0),glm.vec3(55,2.2,-57));
        
        
        this.trans_obelisk = glm.translate(glm.mat4(1.0),glm.vec3(3,0,11));
        //===========================================================
        
        //===========================================================
        //Scales
        this.floor_scale = glm.scale(glm.mat4(1.0),glm.vec3(2000,0,2000));
        this.inner_floor_scale = glm.scale(glm.mat4(1.0),glm.vec3(this.track_length-8,0,this.track_width-8));
        
        this.track_scale = glm.scale(glm.mat4(1.0),glm.vec3(this.track_length,0,this.track_width));
        
        this.sun_scale = glm.scale(glm.mat4(1.0),glm.vec3(50,50,50));
        
        this.lake_scale = glm.scale(glm.mat4(1.0),glm.vec3(9,0.1,7));
        
        this.oasis_scale = glm.scale(glm.mat4(1.0),glm.vec3(12.5,0.1,15));
        
        this.bigpyramid1_scale = glm.scale(glm.mat4(1.0),glm.vec3(100,75,100));
        this.bigpyramid2_scale = glm.scale(glm.mat4(1.0),glm.vec3(150,115,150));
        this.bigpyramid3_scale = glm.scale(glm.mat4(1.0),glm.vec3(100,75,100));
        
        this.leftdoor_scale = glm.scale(glm.mat4(1.0),glm.vec3(2,2,0.2));
        this.rightdoor_scale = glm.scale(glm.mat4(1.0),glm.vec3(0.2,2,2));
        
        this.column_scale = glm.scale(glm.mat4(1.0),glm.vec3(1.5,2.2,1.5));
        
        this.tunnel_column_scale = glm.scale(glm.mat4(1.0),glm.vec3(1/1.5,2.2,1));
        
        
        this.half_scale = glm.scale(glm.mat4(1.0),glm.vec3(0.5,0.5,0.5));
        //===========================================================
        this.lamps_trans = [];
        this.lamps = [];
        
        var step = 2 * Math.PI / 6.0;

        for (let i = 0; i < 6; i++) {
            const lamp = new lampPost(this.gl, [0.190, 0.178, 0.154], [0.480, 0.461, 0.422]);

            const angle = i * step;
            const x = (this.track_length/2+5) * Math.cos(angle);
            const z = (this.track_width/2+5) * Math.sin(angle);

            const rot = glm.toMat4(glm.angleAxis(-angle + Math.PI / 2, glm.vec3(0, 1, 0)));
            const trans = glm.translate(glm.mat4(1.0), glm.vec3(x, 0, z));
            const transform = trans['*'](rot);
            
            this.lamps_trans.push(transform);

            lamp.setShader(this.shaderprog); 
            
            lamp.setTexture(this.lampPostTexture, this.lampTexture);
            
            this.lamps.push(lamp);
        }
        
        
        this.lamps_trans2 = [];
        this.lamps2 = [];
        
        step = 2*Math.PI / 6.0;

        for (let i = 0; i < 6; i++) {
            const lamp = new lampPost(this.gl, [0.190, 0.178, 0.154], [0.480, 0.461, 0.422]);

            const angle = i * step;
            const x = (this.track_length/2-10) * Math.cos(angle-Math.PI/4);
            const z = (this.track_width/2-10) * Math.sin(angle-Math.PI/4);

            const rot = glm.toMat4(glm.angleAxis(-angle + Math.PI / 2, glm.vec3(0, 1, 0)));
            const trans = glm.translate(glm.mat4(1.0), glm.vec3(x, 0, z));
            const transform = trans['*'](rot);
            
            this.lamps_trans2.push(transform);

            lamp.setShader(this.shaderprog); 
            
            lamp.setTexture(this.lampPostTexture, this.lampTexture);
            
            this.lamps2.push(lamp);
        }
        
        //===========================================================        
        
        this.startTime = Date.now(); 
  
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
    }
    
    keyprocess(evt){
        var step=1;
        console.log('Key pressed. Keycode:'+evt.keyCode + ' Char: \'' + String.fromCharCode(evt.charCode)+'\'');
         switch (evt.keyCode) {
             case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
             case 118: // v
                 if(this.mode == 0){
                     this.mode = 1; //Change to Passenger
                 }
                 else if(this.mode == 1){
                     this.mode = 0; //Change to Visitor
                 }
                 break;
             
             case 98: // b
                 if(this.mode == 0){
                     this.mode = 2; //Change to Passenger
                 }
                 else if(this.mode == 2){
                     this.mode = 0; //Change to Visitor
                 }
                 break;

             
             //Add a step in the orientation of the visitor, since the initial orientation is to +Z
             case 119: // w
                 if (this.mode == 0){
                     this.cameraPos.x += step*Math.cos(glm.radians(this.orientation));
                     this.cameraPos.z += step*Math.sin(glm.radians(this.orientation));
                 }
                 break;
             
             //Remove a step in the orientation of the visitor
             case 115: //s
                 if (this.mode == 0){
                     this.cameraPos.x -= step*Math.cos(glm.radians(this.orientation));
                     this.cameraPos.z -= step*Math.sin(glm.radians(this.orientation));
                }
                 break;

             //Change orientation
             case 97: //a
                 if (this.mode == 0){
                     this.orientation-=2;
                 }
                 break;
             
             case 100: //d
                 if (this.mode == 0){
                     this.orientation+=2;
                 }
                 break;
             
             //Switch wheels shpae by initializing a new train with the desired shape (controlled with an if statement in the train constructor)
             case 32: //switch shape
                 if (this.shape == 0){
                    this.shape = 1;
                    this.train = new train(this.gl, [0.0, 0.0, 0.0], [0.450, 0.00450, 0.0193],this.shape);
                    this.train.setShader(this.shaderprog);
                 }
                 else if (this.shape == 1){
                     this.shape = 2;
                     this.train = new train(this.gl, [0.0, 0.0, 0.0], [0.450, 0.00450, 0.0193],this.shape);
                     this.train.setShader(this.shaderprog);
                 }
                 else if (this.shape == 2){
                     this.shape = 0;
                     this.train = new train(this.gl, [0.0, 0.0, 0.0], [0.450, 0.00450, 0.0193],this.shape);
                     this.train.setShader(this.shaderprog);
                 }
             
                 this.engineTexture = new CGRAtexture(this.gl);
                this.engineTexture.load("Textures/locomotive_metal2.jpg");

                this.bodyTexture = new CGRAtexture(this.gl);
                this.bodyTexture.load("Textures/train_metal.jpg");

                this.cgraTexture = new CGRAtexture(this.gl);
                this.cgraTexture.load("Textures/cgra_texture2.jpg");

                this.wheelTexture = new CGRAtexture(this.gl);
                this.wheelTexture.load("Textures/locomotive_metal2.jpg");

                this.train.setTexture(this.engineTexture, this.bodyTexture, this.wheelTexture, this.cgraTexture);
                 break;
        
             case 108: //l  change lights
             break;
             
             
             case 110: //n  night mode
             //Background color
             this.gl.clearColor(0.0952, 0.0989, 0.170, 0.9);
             
             this.sun.setTexture(this.moonTexture);
             
             this.trans_moon_camel = glm.translate(glm.mat4(1.0),glm.vec3(400,226,700));
             break;
             
             
             case 109: //m  day mode
             
             //Background color
             this.gl.clearColor(0, 0, 1, 0.2);
             
             this.sun.setTexture(this.sunTexture);
             
             this.trans_moon_camel = glm.translate(glm.mat4(1.0),glm.vec3(400,-226,700));
             break;
         }
    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT);
        
        
        //Calculate time cycle and view matrix based on mode 
        var time = (Date.now()-this.startTime)/1000.0;  
        var cycleTime = (time % (this.loopDuration + this.pauseDuration)); //Makes sure time is reset after a cycle and isnt infinite

        //Start the cycle stopped
        if (cycleTime < this.pauseDuration) {
           var t = 0;
        } else {
           t = cycleTime - this.pauseDuration;
        }

        var x = (this.track_length/2) * Math.cos(1/this.loopSpeed*t-Math.PI/2);
        var z = (this.track_width/2) * Math.sin(1/this.loopSpeed*t-Math.PI/2);
        
        //Select view based on mode
        if(this.mode == 1){   
            this.viewM = glm.lookAt(glm.vec3(x,1,z),glm.vec3(x+Math.cos(1/this.loopSpeed*t-Math.PI/2),1,z+Math.sin(1/this.loopSpeed*t-Math.PI/2)),glm.vec3(0,1,0));
        }
        else if(this.mode == 0){
            this.viewM = glm.lookAt(this.cameraPos,glm.vec3(this.cameraPos.x+Math.cos(glm.radians(this.orientation)),1,this.cameraPos.z+Math.sin(glm.radians(this.orientation))),glm.vec3(0,1,0));
        }
        else if(this.mode == 2){
            this.viewM = glm.lookAt(glm.vec3((this.track_length/2-5) * Math.cos(1/this.loopSpeed*t-Math.PI/2),1,(this.track_width/2-5) * Math.sin(1/this.loopSpeed*t-Math.PI/2)),glm.vec3(0,1,0),glm.vec3(0,1,0));
        }
        
        //===========================================================
         //Doors
        this.leftdoor.setModelTransformation(this.trans_leftdoor['*'](this.leftdoor_scale));
        this.leftdoor.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.rightdoor.setModelTransformation(this.trans_rightdoor['*'](this.rightdoor_scale));
        this.rightdoor.draw(this.viewM, this.projectionM, this.shaderprog);
        
        //Columns
        this.column1.draw(this.viewM, this.projectionM, this.trans1_column['*'](this.column_scale));
        this.column2.draw(this.viewM, this.projectionM, this.trans2_column['*'](this.column_scale));
    
        this.column3.draw(this.viewM, this.projectionM, this.trans3_column['*'](this.column_scale));
        this.column4.draw(this.viewM, this.projectionM, this.trans4_column['*'](this.column_scale));
        this.column5.draw(this.viewM, this.projectionM, this.trans5_column['*'](this.column_scale));
    
        this.column6.draw(this.viewM, this.projectionM, this.trans6_column['*'](this.column_scale));
        this.column7.draw(this.viewM, this.projectionM, this.trans7_column['*'](this.column_scale));
        this.column8.draw(this.viewM, this.projectionM, this.trans8_column['*'](this.column_scale));
        
        //Tunnel columns
        this.tunnel_column1.draw(this.viewM, this.projectionM, this.trans1_tunnel_column['*'](this.tunnel_column_scale));
        this.tunnel_column2.draw(this.viewM, this.projectionM, this.trans2_tunnel_column['*'](this.tunnel_column_scale));
        this.tunnel_column3.draw(this.viewM, this.projectionM, this.trans3_tunnel_column['*'](this.tunnel_column_scale));
        this.tunnel_column4.draw(this.viewM, this.projectionM, this.trans4_tunnel_column['*'](this.tunnel_column_scale));
        
        
        //Bench
        this.bench1.draw(this.viewM, this.projectionM, this.trans1_bench);
        this.bench2.draw(this.viewM, this.projectionM, this.trans2_bench['*'](glm.toMat4(glm.angleAxis(glm.radians(180.0),glm.vec3(0.0,1.0,0.0)))));
        
        this.bench3.draw(this.viewM, this.projectionM, this.trans3_bench);
        this.bench4.draw(this.viewM, this.projectionM, this.trans4_bench['*'](glm.toMat4(glm.angleAxis(glm.radians(180.0),glm.vec3(0.0,1.0,0.0)))));
        
        this.bench5.draw(this.viewM, this.projectionM, this.trans5_bench);
        this.bench6.draw(this.viewM, this.projectionM, this.trans6_bench['*'](glm.toMat4(glm.angleAxis(glm.radians(180.0),glm.vec3(0.0,1.0,0.0)))));
        
        this.bench7.draw(this.viewM, this.projectionM, this.trans7_bench);
        this.bench8.draw(this.viewM, this.projectionM, this.trans8_bench['*'](glm.toMat4(glm.angleAxis(glm.radians(180.0),glm.vec3(0.0,1.0,0.0)))));
        
        //Track and floor
        this.sphynx.setModelTransformation(this.trans_sphynx);
        this.sphynx.draw(this.viewM, this.projectionM, this.trans_sphynx);
        
        this.floor.setModelTransformation(this.floor_scale);
        this.floor.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.inner_floor.setModelTransformation(this.trans_inner_floor['*'](this.inner_floor_scale));
        this.inner_floor.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.track.setModelTransformation(this.trans_track['*'](this.track_scale));
        this.track.draw(this.viewM, this.projectionM, this.shaderprog);
        
        
        //Pyramids
        this.bigpyramid1.setModelTransformation(this.trans_bigpyramid1['*'](this.bigpyramid1_scale));
        this.bigpyramid1.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.bigpyramid2.setModelTransformation(this.trans_bigpyramid2['*'](this.bigpyramid2_scale));
        this.bigpyramid2.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.bigpyramid3.setModelTransformation(this.trans_bigpyramid3['*'](this.bigpyramid3_scale));
        this.bigpyramid3.draw(this.viewM, this.projectionM, this.shaderprog);
        
        
        //Sun 
        this.sun.setModelTransformation(this.trans_sun['*'](this.sun_scale));
        this.sun.draw(this.viewM, this.projectionM, this.shaderprog);
        
        
        //lakes   
        this.lake.setModelTransformation(this.trans_lake['*'](this.lake_scale));
        this.lake.draw(this.viewM, this.projectionM, this.shaderprog);
        this.oasis.setModelTransformation(this.trans_oasis['*'](this.oasis_scale));
        this.oasis.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.lake2.setModelTransformation(this.trans_lake2['*'](this.lake_scale));
        this.lake2.draw(this.viewM, this.projectionM, this.shaderprog);
        this.oasis2.setModelTransformation(this.trans_oasis2['*'](this.oasis_scale));
        this.oasis2.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.lake3.setModelTransformation(this.trans_lake3['*'](this.lake_scale));
        this.lake3.draw(this.viewM, this.projectionM, this.shaderprog);
        this.oasis3.setModelTransformation(this.trans_oasis3['*'](this.oasis_scale));
        this.oasis3.draw(this.viewM, this.projectionM, this.shaderprog);
        
        //Lamps
        this.lamp1.draw(this.viewM, this.projectionM, this.trans_lamp1['*'](this.half_scale));
        this.lamp2.draw(this.viewM, this.projectionM, this.trans_lamp2['*'](this.half_scale));
        
        for (let i = 0; i < this.lamps.length; i++) {
            this.lamps[i].draw(this.viewM, this.projectionM, this.lamps_trans[i]);
        }
        
        for (let i = 0; i < this.lamps2.length; i++) {
            this.lamps2[i].draw(this.viewM, this.projectionM, this.lamps_trans2[i]);
        }
        
        //Camel
        this.camel.draw(this.viewM, this.projectionM, this.trans_camel,time);
        this.camel2.draw(this.viewM, this.projectionM, this.trans_camel2['*'](this.half_scale), 8);
        this.camel3.draw(this.viewM, this.projectionM, this.trans_camel3['*'](glm.toMat4(glm.angleAxis(glm.radians(135), glm.vec3(0.0, 1.0, 0.0)))),22);
        
        this.camel4.draw(this.viewM, this.projectionM, this.trans_camel4['*'](this.half_scale), 0);
        this.camel5.draw(this.viewM, this.projectionM, this.trans_camel5['*'](glm.toMat4(glm.angleAxis(glm.radians(180), glm.vec3(0.0, 1.0, 0.0)))),0);
        this.camel6.draw(this.viewM, this.projectionM, this.trans_camel6['*'](this.half_scale), 0);
        this.camel7.draw(this.viewM, this.projectionM, this.trans_camel7['*'](glm.toMat4(glm.angleAxis(glm.radians(-75), glm.vec3(0.0, 1.0, 0.0)))),time);
        this.camel8.draw(this.viewM, this.projectionM, this.trans_camel8['*'](glm.toMat4(glm.angleAxis(glm.radians(130), glm.vec3(0.0, 1.0, 0.0)))), 8+time);
        
        this.moon_camel.draw(this.viewM, this.projectionM, this.trans_moon_camel['*'](glm.scale(glm.mat4(1.0),glm.vec3(2,2,2))));
        
        //Turnstile
        this.turnstile1.draw(this.viewM, this.projectionM, this.trans1_turnstile['*'](glm.toMat4(glm.angleAxis(glm.radians(90), glm.vec3(0.0, 1.0, 0.0))))['*'](this.half_scale),time*2);
        this.turnstile2.draw(this.viewM, this.projectionM, this.trans2_turnstile['*'](glm.toMat4(glm.angleAxis(glm.radians(-90), glm.vec3(0.0, 1.0, 0.0))))['*'](this.half_scale),-time*2);
        this.turnstile3.draw(this.viewM, this.projectionM, this.trans3_turnstile['*'](glm.toMat4(glm.angleAxis(glm.radians(-90), glm.vec3(0.0, 1.0, 0.0))))['*'](this.half_scale),-time*2);
        this.turnstile4.draw(this.viewM, this.projectionM, this.trans4_turnstile['*'](glm.toMat4(glm.angleAxis(glm.radians(90), glm.vec3(0.0, 1.0, 0.0))))['*'](this.half_scale),time*2);
        
        //Palmtrees
        
        for (let i = 0; i < this.palmtrees.length; i++) {
            this.palmtrees[i].draw(this.viewM, this.projectionM, this.palmtrees_trans[i]);
        }
        
        //Extra columns
        this.extra_column1.draw(this.viewM, this.projectionM, this.trans1_extra_column['*'](glm.toMat4(glm.angleAxis(glm.radians(90), glm.vec3(1.0, 0.0, 0.0))))['*'](this.column_scale));
        this.extra_column2.draw(this.viewM, this.projectionM, this.trans2_extra_column['*'](this.column_scale));
        this.extra_column3.draw(this.viewM, this.projectionM, this.trans3_extra_column['*'](this.column_scale));
        
        //obelisk
        this.obelisk.draw(this.viewM, this.projectionM, this.trans_obelisk);
        
        //Train
        this.a=(this.track_length/2-2);
        this.b=(this.track_width/2-2);
        this.alpha=1/this.loopSpeed
        
        const I = glm.mat4(1.0);

        //train starts at -pi/2
        const M0 = ellipsePose(this.a, this.b, this.alpha, t, -Math.PI/2);
        const M1 = ellipsePose(this.a, this.b, this.alpha, t,  -2.13*Math.PI/4);
        const M2 = ellipsePose(this.a, this.b, this.alpha, t,  -2.24*Math.PI/4);
        const M3 = ellipsePose(this.a, this.b, this.alpha, t,  -2.38*Math.PI/4);

        this.train.draw(this.viewM, this.projectionM, I, M0, M1, M2, M3, -t);  
        //===========================================================

    }
    
}


var app2 = new myapp2('myCanvas2');

app2.run();

</script>